<a href="https://colab.research.google.com/github/iceangel79/MScWinf_Gruppenarbeit_Sem2/blob/main/Semesterarbeit/onlyAnalysis_Ern%C3%A4hrung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">

# **Personalisierte Ernährung zur Prävention ernährungsbedingter Krankheiten**

<br>

**Gruppe 5**  
Flavia De Solda, Kay Sheppard, Murat Ayhekim, Adrian Martic, Adrian Wegmann  
<br>
FFHS / BFH / HSLU und OST  
Master of Science in Wirtschaftsinformatik  
<br>
**März 2025**

</div>

---

##  Erklärung

Wir erklären hiermit, dass wir die vorliegende Arbeit für das Modul **BINA** bzw. die von uns ausgewiesene Leistung **selbständig**, ohne unzulässige Hilfe Dritter und ausschließlich unter Verwendung der angegebenen Quellen verfasst bzw. erbracht haben.

Zur sprachlichen Überprüfung, zur besseren Verständlichkeit der Zusammenhänge sowie zur Sicherstellung eines konsistenten roten Fadens wurde unterstützend ein KI-Tool eingesetzt.

<div align="right">Zürich, 02.06.2025</div>  
<div align="right">Die unterschriebene Selbständigkeitserklärung befindet sich im Anhang.</div>

---

## Einleitung

Ernährungsbedingte Krankheiten wie Typ-2-Diabetes, Bluthochdruck und Adipositas nehmen weltweit zu. Gleichzeitig bleibt der Zugang zu individueller, präventiv wirksamer Ernährungsberatung beschränkt: Klassische Beratungen sind kostenintensiv und nicht skalierbar, während pauschale Ernährungsempfehlungen oft an den tatsächlichen Bedürfnissen vorbeigehen. Dank moderner Datenanalyse und Machine Learning eröffnen sich heute jedoch neue Möglichkeiten, personalisierte Ernährungsempfehlungen datenbasiert, automatisiert und massentauglich zu gestalten.
Im Rahmen dieser Fallstudie simulieren wir die Entwicklung eines fiktiven digitalen Empfehlungstools namens „NutriDecide“. Dieses Tool soll auf Basis realer Essgewohnheiten, Nährstoffverteilungen und Gesundheitsdaten konkrete Empfehlungen für eine ausgewogene, gesundheitsfördernde Ernährung ableiten.

Wir orientieren uns dabei am **CPA-Framework _"From Data to Decisions"_** und fokussieren uns insbesondere auf folgende Schritte:

1. Zieldefinition und Informationsbedarf  
2. Datenerhebung und -integration  
3. Datenanalyse  
4. Kommunikation der Erkenntnisse für Entscheider

Zwar existieren bereits zahlreiche virtuelle Ernährungstools und -Apps auf dem Markt, jedoch fokussieren sich diese häufig primär auf Gewichtsverlust oder allgemeine Kalorienziele. Eine fundierte, auf wissenschaftlichen Empfehlungen basierende Risikoabschätzung hinsichtlich chronischer Erkrankungen fehlt meist.

NutriDecide setzt genau hier an und verfolgt einen präventiven, individualisierten Ansatz mit klarer Zielsetzung: Die Identifikation gesundheitlich bedenklicher Ernährungsmuster und die Ableitung konkreter Verbesserungsvorschläge auf Basis anerkannter Referenzwerte (z.B. BLV).
Das Tool adressiert den steigenden Bedarf an Gesundheitsprävention und soll potenziell über Krankenkassen angeboten werden. Diese können damit ihren Versicherten einen kostenlosen Zugang zu individuellen Ernährungsempfehlungen bieten – und gleichzeitig durch gezielte Prävention langfristig die Gesundheitskosten senken. „NutriDecide“ positioniert sich damit als innovatives Bindeglied zwischen Versicherer, Nutzer:in und Gesundheitsförderung.


## Theoretische Grundlage
-  Was ist Makronährstoffe: Makronährstoffe sind die grossen Nährstoffe, die unser Körper in grösseren Mengen braucht, im Gegensatz zu Mikronährstoffen wie Vitaminen oder Mineralstoffen.
Wichtigkeit für Körper:
- Ohne Proteine → keine Muskeln, kein Immunsystem.
- Ohne Kohlenhydrate → keine schnelle Energie für Gehirn & Muskeln.
- Ohne Fette → keine fettlöslichen Vitamine, keine Hormone.

---
## 1 Zieldefinition und Informationsbedarf
Damit ein digitales Ernährungstool wie NutriDecide überhaupt sinnvoll entwickelt und eingesetzt werden kann, braucht es zuerst eine solide Datengrundlage.
Dazu gehört nicht nur, zu wissen, was Menschen essen, sondern auch, wie sich diese Essgewohnheiten auf ihre Gesundheit auswirken – und welche Empfehlungen daraus abgeleitet werden können.

Im Sinne des CPA-Frameworks ist es deshalb entscheidend, sich zuerst die richtigen Fragen zu stellen, bevor man technische Lösungen entwickelt.
Nur wenn klar ist, welche Informationen wirklich gebraucht werden, lässt sich ein Tool gestalten, das am Ende für Nutzer:innen oder Organisationen einen echten Mehrwert bringt.

Genau deshalb haben wir vier zentrale Key Analytics Questions (KAQs) formuliert, die unsere Analyse leiten. Sie helfen uns zu verstehen, welche Daten wir brauchen, wie wir sie kombinieren müssen – und was wir daraus lernen wollen.


| **KAQ Nr.** | **Key Analytics Question** | **Daten** | **Vorgehen / Schritte** |
|------------:|----------------------------|-----------|--------------------------|
| 1 | In welchen Ernährungsmustern liegt eine potenziell unausgewogene Nährstoffzufuhr vor? | `diet.csv, examination.csv, questionnaire.csv` | Identifikation von Clustern |
| 2 | Welche Ernährungsmuster sind statistisch signifikant mit bestimmten Krankheitsbildern assoziiert – und lassen sich Risikogruppen identifizieren? | `diet.csv, examination.csv, questionnaire.csv` | Korrelation von Clustern |
| 3 | Wie stark weichen die identifizierten Ernährungsmuster (Cluster/Risikogruppen) von den offiziellen Nährstoffempfehlungen (z. B. BLV) ab – und welche Nährstoffe sind dabei besonders kritisch in Bezug auf präventive Gesundheit? | `diet.csv, examination.csv, questionnaire.csv` | Gruppenvergleich (Mittelwerte), Abweichungsberechnung in %, Heatmap oder Radarplot |
| 4 | Wie können auf Basis individueller Ernährungsmuster und assoziierter Krankheitsrisiken personalisierte Ernährungsempfehlungen abgeleitet werden, um präventiv gegenzusteuern – und welche konkreten Lebensmittel(-gruppen) eignen sich zur Umsetzung? | `diet.csv, examination.csv, questionnaire.csv` | Matching Nährstoffdefizite ↔ Nährstoffreiche Lebensmittel, Scoring, Ranking, einfache Optimierungslogik |

---

##  2 Datenerhebung und -integration

Für die Beantwortung unsere Fragen und dem entwickeln des Tools benötigen, wir Daten welche sich dafür eignen.

**Datenquelle**
Bei der Auswahl unserer Datenquellen haben wir uns an folgenden Kriterien orientiert:

- Die Daten mussten die für unsere Fragestellungen relevanten Informationen enthalten, insbesondere Angaben zur individuellen Nährstoffzufuhr und zum Gesundheitszustand.

- Die Daten sollten eine ausreichende Anzahl von Beobachtungen aufweisen, um statistische Analysen (z.B. Clustering) mit gewisser Aussagekraft durchführen zu können.

- Die Struktur der Datensätze sollte grundsätzlich verständlich und weiterverarbeitbar sein – auch wenn wir eine eigene Bereinigung und Aufbereitung vornehmen würden.

Folgende Datensätze von Kaggle haben unsere Anforderungene erfüllt:
- NHANES-Daten: Ernährungsdaten (diet.csv), Untersuchungsdaten (examination.csv), Diesagnosedaten, Gesundheitsdaten (questionnaire.csv);
- Detailed Meals Macros CLEANED und Food and Nutrition

Für den Abgleich mit offiziellen Nährstoffzufuhrs Empfehlungen greifen wir auf die Richtwerte des Bundesamts für Lebensmittelsicherheit und Veterinärwesen (BLV) zurück. Diese gelten als anerkannter Standard in der Schweiz und kommen in der Gesundheitsförderung sowie Ernährungsberatung zum Einsatz. Im Vergleich mit internationalen Richtwerten der WHO stimmen die grundlegenden Zielwerte weitgehend überein.

 **Verwendete Datensätze**

1. **National Health and Nutrition Examination-Survey (NHANES)-Daten** (Kaggle)
   - Ernährungsdaten (diet.csv): Kalorien, Zucker, Fette, Vitamine
   - Untersuchungsdaten (examination.csv): Blutdruck, Körpergrösse, Gewicht, BMI
   - Fragebogen (questionnaire.csv): Diagnosedaten, Gesundheitsverhalten

   https://www.kaggle.com/datasets/cdc/national-health-and-nutrition-examination-survey

2. **Detailed Meals Macros CLEANED** (Kaggle)  
   - Personenbezogene Merkmale: Alter, Geschlecht, Grösse, Gewicht, Aktivitätslevel  
   - Nährwertprofile der Mahlzeiten (Frühstück, Mittagessen, Abendessen, Snacks)  
   - Krankheitsstatus (z. B. Hypertension, Diabetes, Weight Gain)

   https://www.kaggle.com/code/chaitanya99/eda-nutritional-patterns-dite-recommendation/input?scriptVersionId=211165422&select=detailed_meals_macros_CLEANED.csv

3. **Food and Nutrition Dataset** (Kaggle)  
   - Nährwerte einzelner Lebensmittel (Makros & Mikros)  
   - Ermöglicht detaillierte Aufschlüsselung komplexer Mahlzeiten

   https://www.kaggle.com/datasets/utsavdey1410/food-nutrition-dataset/data

4. **BLV Schweizer Referenzwerte**  
   - Offizielle Empfehlungen zur idealen Nährstoffzufuhr

   https://www.blv.admin.ch/blv/de/home/lebensmittel-und-ernaehrung/ernaehrung/empfehlungen-informationen/naehrstoffe/naehrstoffzufuhr-dynamische-tabelle.html



**Datenqualität**
Die von uns verwendeten Datensätze stammen aus verschiedenen öffentlich zugänglichen Quellen (z. B. Kaggle oder offizielle Webseiten) und unterscheiden sich hinsichtlich Struktur, Umfang und Granularität.
Der Datensatz „detailed_meals_macros_CLEANED.csv“ enthält 1’697 Einträge zu individuellen Tagesmahlzeiten inklusive Angabe von möglichen Erkrankungen.
Der Datensatz „Food_and_Nutrition__.csv“ umfasst 1’698 Lebensmittelprofile mit Angaben zu Makro- und Mikronährstoffen.
Zusätzlich wurden fünf Dateien aus einer erweiterten Lebensmitteldatenbank (FOOD-DATA-GROUP1 bis GROUP5) kombiniert, welche in Summe 2’395 Einträge enthalten.

Die Datensätze wiesen bei erster Sichtung eine grundsätzlich saubere Struktur auf. Trotzdem war eine eigenständige Prüfung und Bereinigung notwendig. Vor allem im Ernährungsdatensatz mussten wir sicherstellen, dass alle relevanten Spalten vollständig befüllt und keine Duplikate vorhanden waren.
Auch bei den Nährwertangaben war eine Überprüfung auf Konsistenz und Redundanz notwendig.
Die BLV-Referenzwerte wurden als externe Normquelle verwendet. Diese blieben inhaltlich unverändert, wurden jedoch bezüglich Formatierung und Spaltenstruktur angepasst, um sie in die Analyse zu integrieren.

- **Datenbereinigung:**
Zur Bereinigung der Ernährungsdaten (detailed_meals_macros_CLEANED.csv) wurden zunächst einzelne fehlende Werte in kritischen Spalten überprüft. Es wurde eine Zeile entfernt, in der der Wert für „Breakfast Carbohydrates“ fehlte.
Anschliessend wurden Duplikate bereinigt, wobei die Spalte Disease_List ausgenommen wurde, um unterschiedliche Krankheitsprofile nicht zu verlieren.

Die Krankheitsangaben wurden in strukturierte Listen umgewandelt, sodass auch Mehrfacherkrankungen korrekt berücksichtigt werden konnten. Daraus wurde eine zusätzliche Spalte Disease_Count (Anzahl Krankheiten pro Person) sowie ein Disease_Tuple für Gruppierungen erstellt.

Im Datensatz „Food_and_Nutrition__.csv“ waren keine fehlenden Werte vorhanden. Es wurden ausschliesslich Dubletten entfernt.
Bei den kombinierten Lebensmitteldaten (GROUP1–GROUP5) verhielt es sich gleich: Es gab keine fehlenden Einträge, jedoch wurden Doppelerfassungen eliminiert. Der kombinierte Datensatz bestand am Ende aus 2’395 vollständigen Einträgen.

Die BLV-Daten wurden ausschliesslich formatiert (Spaltennamen bereinigt, leere Zeilen entfernt), jedoch inhaltlich nicht verändert, da sie als offizielle Referenz gelten und keine Manipulation vorgenommen werden sollte.
- **Feature Engineering:** Erstellung neuer Features, die für die Analyse relevant sind (z.B. Mikronährstoffe, Kalorien).



---

##  3 Datenanalyse

**1. Explorative Datenanalyse (EDA)**
- **Deskriptive Statistiken:** Berechnung von Mittelwerten, Medianen und Standardabweichungen für die Mikronährstoffprofile.
- **Visualisierungen:** Erstellung von Diagrammen (Boxplots, Histogramme) zur Visualisierung der Verteilung der Nährstoffe.

**2. Ernährungsmuster identifizieren**
- **Clusteranalyse:** Verwendung von Algorithmen wie K-Means oder Hierarchical Clustering, um Ernährungsmuster zu identifizieren.
- **Interpretation der Cluster:** Analysiere die Cluster, um typische Ernährungsmuster zu beschreiben.

**3. Zusammenhang zwischen Ernährung und Krankheitsrisiken**
- **Korrelationsanalyse:** Untersuchung der Korrelation zwischen Ernährungsmustern und Krankheitsrisiken.
- **Regressionsmodelle:** Erstelle Regressionsmodelle, um den Einfluss der Ernährung auf Krankheitsrisiken zu quantifizieren.

**4. Identifikation unausgewogener Ernährungsmuster**
- **Vergleich mit Empfehlungen:** Vergleiche der identifizierten Ernährungsmuster mit offiziellen Ernährungsempfehlungen.
- **Identifikation von Defizien/Überschüssen:** Bestimmung, welche Muster potenziell unausgewogen sind (z.B. zu wenig Protein, zu viel Zucker).

**5. Vergleich von Personen mit und ohne Krankheit**
- **Gruppenvergleich:** Vergleich der Ernähungsprofile von Personen mit und ohne Krankheiten.
- **Statistische Tests:** Verwendung von Tests wie dem t-Test, um signifikante Unterschiede zu identifizieren.

**6. Empfehlungen basierend auf Clusterbildung**
- **Personalisierte Empfehlungen:** Erstellung von personalisierten Ernährungsempfehlungen basierend auf den identifizierten Clustern.
- **Berücksichtigung von Krankheiten:** Integration von offiziellen Empfehlungen für bestimmte Krankheiten in die personalisierten Empfehlungen.


---

## 4 Kommunikation der Erkenntnisse für Entscheider

- Visualisierung der Resultate (Dashboards, Diagramme)  
- Empfehlungen je Zielgruppe  
- Ableitung konkreter Maßnahmen oder Produktideen

---


# Wir wollen dich zuerst kennenlernen. Berehcne deine Tagesbedarf

🔍 Persönliche Angaben:


In [ ]:
#pip install scikit-learn
#pip install kaggle

In [3]:
# Datenverarbeitung und Analyse
import pandas as pd
import numpy as np

# Visualisierung
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Preprocessing & Reduktion
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Clustering
from sklearn.cluster import KMeans

# Klassifikation & Modellbewertung
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Interaktive Umgebung (optional, z. B. in Jupyter Notebooks)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MarkDown
from IPython.display import display, Markdown



In [4]:
def get_user_inputs():
    print("🔍 Persönliche Angaben:")
    gewicht = float(input("Gewicht (kg): "))
    groesse = float(input("Grösse (cm): "))
    alter = int(input("Alter (Jahre): "))
    geschlecht = input("Geschlecht (m/w): ").lower()

    print("\n⚡ Aktivitätslevel:")
    print("1 = Keine Bewegung (Bürojob)")
    print("2 = Wenig aktiv (1–2x pro Woche Sport)")
    print("3 = Mässig aktiv (3–5x pro Woche Sport)")
    print("4 = Sehr aktiv (tägliches Training)")
    print("5 = Extrem aktiv (körperlich harte Arbeit)")
    aktivitaetslevel = int(input("Wähle dein Aktivitätslevel (1–5): "))

    return gewicht, groesse, alter, geschlecht, aktivitaetslevel

def berechne_umsatz(gewicht, groesse, alter, geschlecht, aktivitaetslevel):
    # Grundumsatz berechnen (Mifflin-St. Jeor-Formel)
    if geschlecht == 'm':
        grundumsatz = 10 * gewicht + 6.25 * groesse - 5 * alter + 5
    elif geschlecht == 'w':
        grundumsatz = 10 * gewicht + 6.25 * groesse - 5 * alter - 161
    else:
        print("❌ Ungültiges Geschlecht.")
        return None, None, None

    aktivitaetsfaktor = {
        1: 1.2,
        2: 1.375,
        3: 1.55,
        4: 1.725,
        5: 1.9
    }

    faktor = aktivitaetsfaktor.get(aktivitaetslevel, 1.2)
    gesamtumsatz = grundumsatz * faktor
    leistungsumsatz = gesamtumsatz - grundumsatz

    return grundumsatz, leistungsumsatz, gesamtumsatz

# Ablauf
gewicht, groesse, alter, geschlecht, aktivitaetslevel = get_user_inputs()
grundumsatz, leistungsumsatz, gesamtumsatz = berechne_umsatz(gewicht, groesse, alter, geschlecht, aktivitaetslevel)

if grundumsatz:
    print("\n📊 Ergebnisse:")
    print(f"- Grundumsatz (BMR): {round(grundumsatz)} kcal")
    print(f"- Leistungsumsatz:   {round(leistungsumsatz)} kcal")
    print(f"- Gesamtumsatz (TDEE): {round(gesamtumsatz)} kcal")


🔍 Persönliche Angaben:
Gewicht (kg): 70
Grösse (cm): 177
Alter (Jahre): 37
Geschlecht (m/w): m

⚡ Aktivitätslevel:
1 = Keine Bewegung (Bürojob)
2 = Wenig aktiv (1–2x pro Woche Sport)
3 = Mässig aktiv (3–5x pro Woche Sport)
4 = Sehr aktiv (tägliches Training)
5 = Extrem aktiv (körperlich harte Arbeit)
Wähle dein Aktivitätslevel (1–5): 3

📊 Ergebnisse:
- Grundumsatz (BMR): 1626 kcal
- Leistungsumsatz:   894 kcal
- Gesamtumsatz (TDEE): 2521 kcal


In [5]:
def get_user_inputs():
    gewicht = float(input("Gewicht (kg): "))
    groesse = float(input("Grösse (cm): "))
    alter = int(input("Alter: "))
    kalorien = int(input("Kalorienzufuhr (kcal): "))
    fett = int(input("Fett (g): "))
    zucker = int(input("Zucker (g): "))
    natrium = int(input("Natrium (mg): "))
    return gewicht, groesse, alter, kalorien, fett, zucker, natrium

def berechne_und_empfehle(gewicht, groesse, alter, kalorien, fett, zucker, natrium):
    bmi = gewicht / (groesse / 100) ** 2

    print("++++++++++" + str(gesamtumsatz) + "++++++++")
    zu_viel_kcal = kalorien > gesamtumsatz #2500
    zu_viel_fett = fett > 75
    zu_viel_zucker = zucker > 50
    zu_viel_natrium = natrium > 2000

    if zu_viel_kcal and zu_viel_fett:
        muster = 'Fett- & kalorienreich'
    elif zu_viel_zucker:
        muster = 'Zuckerreich'
    elif zu_viel_natrium:
        muster = 'Salzreich'
    else:
        muster = 'Unauffällig'

    lebensmittel_empfehlungen = {
        'Fett- & kalorienreich': {
            'Empfohlen': "Brokkoli, Kartoffeln, Vollkornreis, Hülsenfrüchte, Lachs",
            'Meiden': "Fast Food, Wurst, Käse, Frittierte Speisen"
        },
        'Zuckerreich': {
            'Empfohlen': "Beeren, Äpfel, Wasser, Ungesüsster Tee",
            'Meiden': "Softdrinks, Süssigkeiten, gezuckertes Müsli"
        },
        'Salzreich': {
            'Empfohlen': "Frisches Gemüse, Spinat, Banane, ungesalzene Nüsse",
            'Meiden': "Chips, Fertiggerichte, Wurst, Instantnudeln"
        },
        'Unauffällig': {
            'Empfohlen': "Ausgewogene Mischkost – Obst, Gemüse, Vollkorn",
            'Meiden': "Keine besonderen Einschränkungen"
        }
    }

    emp = lebensmittel_empfehlungen[muster]['Empfohlen']
    mei = lebensmittel_empfehlungen[muster]['Meiden']

    print(f"\n🔎 Ergebnis für deine Eingaben:")
    print(f"- BMI: {bmi:.1f}")
    print(f"- Ernährungsmuster: {muster}")
    print(f"\n✅ Empfohlene Lebensmittel:\n{emp}")
    print(f"\n⛔ Zu meiden:\n{mei}")

# Ausführung
werte = get_user_inputs()
berechne_und_empfehle(*werte)


Gewicht (kg): 70
Grösse (cm): 177
Alter: 37
Kalorienzufuhr (kcal): 2000
Fett (g): 60
Zucker (g): 60
Natrium (mg): 0
++++++++++2520.6875++++++++

🔎 Ergebnis für deine Eingaben:
- BMI: 22.3
- Ernährungsmuster: Zuckerreich

✅ Empfohlene Lebensmittel:
Beeren, Äpfel, Wasser, Ungesüsster Tee

⛔ Zu meiden:
Softdrinks, Süssigkeiten, gezuckertes Müsli


**Überblick über das Thema Ernährung und die Krankheiten Bluthochdruck, Diabetes und Übergewicht**

<img src="https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/1a912ddf00236ad8302bc8cc1fbfa6225ad52127/Semesterarbeit/Images/Ernährung_und_Ges.png" alt="Beispielbild" width="300" />

In [ ]:
# =====================================
# Author:
# KAQ:
# Description:
# =====================================

# 1. CSV-Dateien laden
diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv")
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

# 2. Relevante Spalten auswählen
diet_cols = ['SEQN', 'DR1TKCAL', 'DR1TSUGR', 'DR1TTFAT', 'DR1TSODI']  # Kalorien, Zucker, Fett, Natrium
examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']  # BMI, Blutdruck
questionnaire_cols = ['SEQN', 'DIQ010']  # Frage: Hat ein Arzt Diabetes diagnostiziert?

diet_small = diet[diet_cols]
examination_small = examination[examination_cols]
questionnaire_small = questionnaire[questionnaire_cols]

# 3. Zusammenführen über SEQN
df = diet_small.merge(examination_small, on="SEQN", how="inner")
df = df.merge(questionnaire_small, on="SEQN", how="inner")

# 4. Zeilen mit fehlenden Werten entfernen und Kopie erstellen
df_clean = df.dropna().copy()  # <- .copy() vermeidet SettingWithCopyWarning

# 5. Gesundheitsstatus klassifizieren
def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Gesund'

# Saubere Zuweisung ohne Warnung
df_clean.loc[:, 'Gesundheitsstatus'] = df_clean.apply(classify_health, axis=1)

# 6. Optional: Ergebnisse speichern oder anzeigen
df_clean.to_csv("NHANES_bereinigt.csv", index=False)
print(df_clean.head())

**Zusammenfassung der Datenbereinigung und Klassifikation (NHANES-Projekt)**

**Ziel:**

Vorbereitung der NHANES-Daten zur Analyse des Zusammenhangs zwischen **Ernährung** und **Gesundheitszustand**: Diabetes, Bluthochdruck, Übergewicht und "Gesund".



---


**Durchgeführte Bereinigungsschritte**

**1. Relevante Spalten extrahiert**

Aus den umfangreichen Rohdaten (jeweils hunderte Spalten) wurden nur die **für die Analyse relevanten Variablen** ausgewählt:

- diet.csv: Kalorien, Zucker, Fett, Natrium
- examination.csv: BMI, systolischer & diastolischer Blutdruck
- questionnaire.csv: Diabetesdiagnose (Selbstauskunft)

--> Reduktion auf ca. 8 Kernvariablen + SEQN.


---



**2. Zusammenführung über SEQN**

Alle drei Dateien wurden anhand der eindeutigen Teilnehmer-ID SEQN zusammengeführt.

--> Dadurch entsteht ein vollständiger Datensatz pro Person.


---



**3. Bereinigung unvollständiger Fälle**

Zeilen mit fehlenden Werten in relevanten Variablen wurden entfernt (dropna() + .copy()).

--> Ergebnis: nur vollständige, auswertbare Datensätze.


---



**Klassifikation in Gesundheitsgruppen**

🔹 **Genutzte Kriterien pro Gesundheitsmerkmal:**

| Gesundheitsmerkmal | Quelle            | Variable(n)                   | Klassifikationskriterium                                                |
|--------------------|-------------------|-------------------------------|--------------------------------------------------------------------------|
| Diabetes           | `questionnaire.csv` | `DIQ010`                      | Arzt hat Diabetes diagnostiziert (`== 1`)                               |
| Bluthochdruck      | `examination.csv`   | `BPXSY1`, `BPXDI1`            | systolisch ≥ 130 oder diastolisch ≥ 80 mmHg                             |
| Übergewicht        | `examination.csv`   | `BMXBMI`                      | BMI ≥ 25                                                                 |
| Gesund             | —                 | abgeleitet                    | Wenn keines der oben genannten Kriterien erfüllt ist                    |



**Wichtig:**

Für Bluthochdruck und Übergewicht wurde **ausschliesslich auf objektive Messwerte** aus examination.csv zurückgegriffen.
Die Fragebogendaten wurden hier **nicht verwendet**, da sie redundant oder subjektiv wären.

> ### Zusätzliche Definition **"Gesund"**
>
> **"Gesund" heisst nicht:**
> - dass die Person rundum medizinisch gesund ist  
> - oder dass sie keinen Risikofaktor für andere Krankheiten aufweist
>
> **"Gesund" heisst nur:**  
> keine der drei gewählten Bedingungen erfüllt:
> - **BMI** < 25  
> - **Systolisch** < 130 mmHg **und** **Diastolisch** < 80 mmHg  
> - keine Angabe **„Diabetes diagnostiziert“**


---



**Zwei Klassifikationsformen möglich:**

**1. Hauptgruppe (exklusiv):**
- Jede Person gehört zu einer von vier Gruppen:
  - Diabetes > Bluthochdruck > Übergewicht > Gesund (Prioritätsreihenfolge)

**2. Mehrfachklassifikation (erweiterbar):**
- Für jede Person wird zusätzlich gespeichert:
  - hat_diabetes: ja/nein
  - hat_bluthochdruck: ja/nein
  - hat_übergewicht: ja/nein
  - ist_gesund: ja/nein (wenn keine der anderen Merkmale zutrifft)

--> Dies erlaubt flexiblere Filterung, z. B. „Zeige alle mit Diabetes UND Übergewicht“.


---



**Ergebnis:**
Ein bereinigter und analysierbarer DataFrame:
- Ohne fehlende Werte
- Mit klassifizierten Gesundheitsgruppen
- Ideal vorbereitet für:
 - Vergleichsanalyse (z. B. Kalorienaufnahme nach Gesundheitsstatus)
 - Visualisierung (Boxplots, Heatmaps etc.)

**1. Häufigkeit von Gesundheitszuständen**

**Erklärung:**

Dieses Balkendiagramm zeigt, wie viele Personen in deinem Datensatz zu jeder Gesundheitsgruppe gehören: **Gesund, Übergewicht, Bluthochdruck, Diabetes**.

--> Es vermittelt ein erstes Gefühl für die Verteilung und Relevanz dieser Zustände in der Bevölkerung.

In [ ]:
plt.figure(figsize=(8, 5))
df_clean['Gesundheitsstatus'].value_counts().plot(kind='bar', color='skyblue')
plt.title("Häufigkeit von Gesundheitszuständen")
plt.ylabel("Anzahl Personen")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Interpretation: Häufigkeit von Gesundheitszuständen**

Die Balkengrafik zeigt die Anzahl der Personen in den vier definierten Gesundheitsgruppen: **Diabetes, Bluthochdruck, Übergewicht** und **"Gesund“** (im Sinne von: keine dieser drei Diagnosen).

**Was sehen wir?**

- Ein grosser Teil der Personen fällt in die Kategorien Übergewicht und Bluthochdruck.

- Die Gruppe der Diabetiker ist kleiner, aber deutlich vertreten.

- Die Gruppe „Gesund“ ist die Referenzgruppe – sie umfasst nur Personen, die keine der drei genannten Kriterien erfüllen.

**Interpretation:**

- Die hohe Anzahl übergewichtiger und bluthochdruckbetroffener Personen deutet darauf hin, dass **ernährungs- und lebensstilbedingte Erkrankungen weit verbreitet** sind.

- Es wird deutlich, dass **nur ein vergleichsweise kleiner Teil der Bevölkerung** als „nicht betroffen“ gilt.

- Allerdings ist zu beachten, dass „Gesund“ in diesem Kontext nur bedeutet, dass **keine der drei spezifischen Bedingungen** (Diabetes, Bluthochdruck, Übergewicht) vorliegt – andere gesundheitliche Einschränkungen sind dabei **nicht ausgeschlossen**.

**Fazit:**

Diese Visualisierung liefert einen klaren Überblick über die Verteilung der Gesundheitszustände und schafft ein Bewusstsein für die Relevanz dieser drei Volkskrankheiten in der Bevölkerung. Sie bildet eine solide Grundlage für die nachfolgenden Analysen zu Ernährungseinflüssen.

**BMI nach Gesundheitsgruppe (Boxplot)**

**Erklärung:**

Diese Boxplots zeigen die BMI-Verteilung (Körpergewicht) in den verschiedenen Gesundheitsgruppen.

--> Man erkennt sofort, dass Übergewichtige einen deutlich höheren BMI haben, während Gesunde im Normalbereich liegen.

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_clean, x='Gesundheitsstatus', y='BMXBMI')
plt.title("BMI-Verteilung nach Gesundheitsgruppe")
plt.ylabel("BMI (kg/m²)")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Interpretation: BMI-Verteilung nach Gesundheitsgruppe**

Diese Boxplot-Grafik veranschaulicht die **Verteilung des Body Mass Index (BMI)** innerhalb der vier Gesundheitsgruppen:

**Keine der drei Diagnosen ("Gesund"), Übergewicht, Bluthochdruck** und **Diabetes.**

**Was sehen wir?**

- Die Gruppe **"Übergewicht“** zeigt erwartungsgemäss die höchsten BMI-Werte mit einem Median deutlich über dem Schwellenwert von 25 kg/m².

- Auch Personen mit Bluthochdruck und Diabetes haben im Mittel erhöhte BMI-Werte – häufig im Bereich von Übergewicht oder Adipositas.

- Die Gruppe „Keine der drei Diagnosen“ weist die niedrigsten BMI-Werte auf und liegt überwiegend im Normalbereich.

**Interpretation:**

- Der BMI unterscheidet sich **signifikant zwischen den Gruppen**, was zeigt:
→ Ein **erhöhter BMI ist nicht nur bei Übergewicht**, sondern auch bei **Diabetes und Bluthochdruck** häufig zu beobachten.

- Die Daten bestätigen den bekannten Zusammenhang zwischen **Übergewicht und dem Risiko für chronische Erkrankungen**.

- Der Boxplot macht zudem deutlich, dass es innerhalb jeder Gruppe **eine gewisse Streuung** gibt – also z. B. auch normalgewichtige Personen mit Bluthochdruck vorkommen können.

**Fazit:**

Diese Visualisierung unterstützt die Annahme, dass **ein hoher BMI ein zentraler Risikofaktor für ernährungsbedingte Krankheiten** wie Bluthochdruck und Diabetes ist. Der Boxplot zeigt anschaulich, wie sich die Körpergewichtswerte über die Gesundheitsgruppen hinweg unterscheiden und unterstreicht die Bedeutung von Gewichtsmanagement in der Prävention.

**3. Zuckeraufnahme nach Gesundheitsstatus**

**Erklärung:**

Dieser Plot zeigt, wie viel **Zucker** die Personen in den verschiedenen Gruppen zu sich nehmen.

--> Besonders interessant ist der Vergleich zwischen **Diabetikern** und **Gesunden** – man erwartet hier eigentlich Unterschiede.

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_clean, x='Gesundheitsstatus', y='DR1TSUGR')
plt.title("Zuckeraufnahme (g) nach Gesundheitsgruppe")
plt.ylabel("Zuckeraufnahme (g)")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Interpretation: Zuckeraufnahme nach Gesundheitsgruppe**

Diese Boxplot-Grafik zeigt, wie viel **Zucker pro Tag (in Gramm)** Personen in den verschiedenen Gesundheitsgruppen zu sich nehmen:

- **Diabetes**

- **Bluthochdruck**

- **Übergewicht**

- **"Gesund" (Keine der drei Diagnosen)**

**Was sehen wir?**

- Die Verteilung der Zuckeraufnahme ist in allen Gruppen relativ breit – von sehr niedrigen bis sehr hohen Werten.

- Überraschend: **Diabetiker konsumieren nicht deutlich weniger Zucker** als andere Gruppen.
→ Der Median (mittlerer Wert) der Zuckeraufnahme ist bei Diabetikern **vergleichbar oder nur leicht niedriger** als bei Gesunden.

- In der Gruppe **"Gesund"** gibt es ebenfalls hohe Zuckermengen, aber auch viele Personen mit niedrigem Konsum.

**Interpretation:**

- Man könnte erwarten, dass **Diabetiker bewusst weniger Zucker konsumieren** – entweder durch ärztliche Beratung oder eigene Vorsicht.

- Die Daten zeigen jedoch, dass dies **nicht zwingend der Fall ist**:
  - Entweder halten sich viele nicht an entsprechende Empfehlungen
  - Oder die Messung (24-Stunden-Ernährungstag) bildet nicht das typische Verhalten ab.

- Auch Menschen ohne Diagnose konsumieren teilweise **Zuckermengen, die als gesundheitlich bedenklich** gelten.

**Fazit:**

Diese Visualisierung macht deutlich, dass **Zuckeraufnahme nicht klar nach Gesundheitsstatus unterscheidbar ist**.
Gerade im Hinblick auf **Prävention von Diabetes und Übergewicht** könnte dies ein Hinweis auf **Verbesserungspotenzial bei Aufklärung und Ernährung** sein.



**4. Korrelation zwischen Ernährung und Gesundheitswerten (Heatmap)**

**Erklärung:**

Diese Heatmap zeigt die **Zusammenhänge zwischen ausgewählten Ernährungsfaktoren** (wie Kalorien, Fett, Zucker und Natrium) und **gesundheitsrelevanten Werten** (BMI und Blutdruck).

--> Z. B. erkennst man, ob **mehr Kalorien oder Fett mit einem höheren BMI** verbunden sind oder **mehr Natrium mit einem erhöhten Blutdruck** zusammenhängt.

Je **dunkler und grösser die Zahl im Feld**, desto **stärker der Zusammenhang**.
Rote Felder bedeuten: positiver Zusammenhang – Blaue Felder: negativer Zusammenhang.

In [ ]:
# Relevante Spalten auswählen
diet_small = diet[['SEQN', 'DR1TKCAL', 'DR1TSUGR', 'DR1TTFAT', 'DR1TSODI']]  # Kalorien, Zucker, Fett, Natrium
examination_small = examination[['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']]     # BMI, Blutdruck
questionnaire_small = questionnaire[['SEQN', 'DIQ010']]                     # Diabetes-Diagnose

# Daten zusammenführen
df = diet_small.merge(examination_small, on="SEQN", how="inner")
df = df.merge(questionnaire_small, on="SEQN", how="inner")

# Bereinigen: nur vollständige Einträge behalten
df_clean = df.dropna().copy()

# Klassifikation: Gesundheitsstatus zuweisen
def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Keine der drei Diagnosen'

df_clean['Gesundheitsstatus'] = df_clean.apply(classify_health, axis=1)

# Variablennamen für bessere Lesbarkeit umbenennen
corr_labels = {
    'DR1TKCAL': 'Kalorien',
    'DR1TTFAT': 'Fett',
    'DR1TSUGR': 'Zucker',
    'DR1TSODI': 'Natrium',
    'BMXBMI': 'BMI',
    'BPXSY1': 'Syst. Blutdruck',
    'BPXDI1': 'Diast. Blutdruck'
}

# Korrelationen berechnen
selected_cols = list(corr_labels.keys())
corr = df_clean[selected_cols].corr()
corr = corr.rename(index=corr_labels, columns=corr_labels)

# Fokus: Ernährung (Zeilen) vs. Gesundheitswerte (Spalten)
ernaehrung = ['Kalorien', 'Fett', 'Zucker', 'Natrium']
gesundheit = ['BMI', 'Syst. Blutdruck', 'Diast. Blutdruck']
corr_subset = corr.loc[ernaehrung, gesundheit]

# Heatmap zeichnen
plt.figure(figsize=(8, 6))
sns.heatmap(corr_subset, annot=True, cmap="RdBu", center=0, linewidths=0.5)
plt.title("Zusammenhang zwischen Ernährung und Gesundheitswerten")
plt.xlabel("Gesundheitswert")
plt.ylabel("Ernährungsmerkmal")
plt.tight_layout()
plt.show()



**Interpretation: Zusammenhang zwischen Ernährung und Gesundheitswerten (Heatmap)**

Diese Heatmap zeigt, **wie stark ausgewählte Ernährungsfaktoren** mit **gesundheitsrelevanten Messwerten** (BMI und Blutdruck) zusammenhängen:

- **Ernährung:** Kalorien, Fett, Zucker, Natrium

- **Gesundheitswerte:** BMI, systolischer & diastolischer Blutdruck

**Was sehen wir?**

- Die **stärksten positiven Zusammenhänge** zeigen sich zwischen:

  - **Kalorien & BMI**

  - **Fett & BMI**

  - **Natrium & systolischem Blutdruck**

- Der **Zusammenhang zwischen Zuckeraufnahme und Gesundheitswerten** ist schwach oder leicht negativ.

- Insgesamt sind die Korrelationen **moderat**, was typisch ist für komplexe Zusammenhänge im Bereich Ernährung und Gesundheit.

**Interpretation:**

- Es zeigt sich, dass **höherer Kalorien- und Fettkonsum tendenziell mit einem höheren BMI** einhergeht.

- **Mehr Natrium (Salz) steht in Verbindung mit einem höheren systolischen Blutdruck**, was bekannte gesundheitliche Risiken widerspiegelt.

- Die Zuckeraufnahme steht **nicht klar im Zusammenhang mit BMI oder Blutdruck** – das kann verschiedene Ursachen haben:

  - Zucker wirkt kurzfristig und ist schwer messbar in einem einzelnen Tagesprotokoll.

  - Es gibt viele individuelle Unterschiede (Bewegung, Stoffwechsel, Medikamente etc.)

**Fazit:**

Diese Visualisierung macht deutlich, dass **einige Ernährungsfaktoren – besonders Kalorien, Fett und Natrium – in einem messbaren Zusammenhang mit gesundheitlichen Werten stehen.**
Sie unterstreicht die Bedeutung einer ausgewogenen Ernährung zur **Vorbeugung von Übergewicht und Bluthochdruck** – und zeigt, dass nicht alle Risikofaktoren sofort offensichtlich sind.

**5. Kalorienaufnahme vs. BMI (Scatterplot)**

**Erklärung:**

Dieses Streudiagramm zeigt den Zusammenhang zwischen **Kalorienaufnahme und Körpergewicht (BMI)**.

--> Man sieht, ob tendenziell höhere Kalorien mit höherem Gewicht einhergehen – und wie sich das nach Gruppen unterscheidet.

In [ ]:
sns.scatterplot(data=df_clean, x='DR1TKCAL', y='BMXBMI', hue='Gesundheitsstatus', palette='Set2')
plt.title("Kalorienaufnahme vs. BMI")
plt.xlabel("Kalorienaufnahme (kcal)")
plt.ylabel("BMI (kg/m²)")
plt.tight_layout()
plt.show()

**Interpretation: Kalorienaufnahme vs. BMI (Scatterplot)**

Dieses Streudiagramm zeigt den **Zusammenhang zwischen der täglichen Kalorienaufnahme** und dem **Körpergewicht (BMI)**.

Jeder Punkt steht für eine Person – farblich gruppiert nach Gesundheitsstatus:

- **"Gesund" (Keine der drei Diagnosen)**

- **Übergewicht**

- **Bluthochdruck**

- **Diabetes**

**Was sehen wir?**

- Es gibt **eine breite Streuung**: Personen mit ähnlicher Kalorienaufnahme haben sehr unterschiedliche BMI-Werte.

- Trotzdem ist ein **leichter Trend erkennbar**:

--> Bei höheren Kalorienwerten häufen sich Personen mit Übergewicht (orange Punkte).

- Die Gruppe **"Gesund" (Keine der drei Diagnosen)** kommt überwiegend bei niedrigeren BMI-Werten vor, aber nicht unbedingt bei geringerer Kalorienaufnahme.

- **Extrem hohe Kalorienwerte** sind selten, aber in allen Gruppen vertreten.

**Interpretation:**

Der Plot zeigt, dass eine **hohe Kalorienaufnahme nicht automatisch zu einem hohen BMI führt**, aber oft **damit einhergeht** – besonders in der Gruppe mit Übergewicht.

Der Zusammenhang ist **nicht linear oder stark ausgeprägt**, was zeigt:

Körpergewicht hängt nicht nur von Kalorien, sondern auch von **Bewegung, Genetik, Stoffwechsel und anderen Faktoren** ab.

Es gibt Menschen mit hohem Kalorienkonsum und dennoch niedrigem BMI (z. B. sportlich aktive).

**Fazit:**

Die Visualisierung verdeutlicht, dass es **einen leichten Zusammenhang zwischen Kalorienzufuhr und Körpergewicht** gibt – aber keine einfache 1:1-Beziehung.
Sie zeigt, wie wichtig es ist, **Ernährung ganzheitlich zu betrachten** – Kalorien sind ein wichtiger Faktor, aber **nicht der einzige**.


---



**Zusammenfassung der Erkenntnisse aus dem Überblick über das Thema Ernährung und die Krankheiten Bluthochdruck, Diabetes und Übergewicht**

Die Analyse zeigt, dass Übergewicht und Bluthochdruck in der untersuchten Bevölkerung besonders häufig auftreten, während nur ein kleiner Anteil als "gesund" (keine der drei Diagnosen) gilt. Diabetes ist seltener, aber ebenfalls präsent. Besonders auffällig ist der Zusammenhang zwischen BMI und Gesundheitsstatus: Personen mit Übergewicht, Bluthochdruck oder Diabetes haben im Schnitt deutlich höhere BMI-Werte als gesunde Personen.

Überraschend ist, dass Diabetiker im Mittel nicht deutlich weniger Zucker konsumieren als Gesunde. Die Zuckeraufnahme variiert stark innerhalb aller Gruppen, was auf unzureichende Umsetzung von Ernährungsempfehlungen oder methodische Einschränkungen hindeuten könnte.

Die Korrelationen zeigen, dass höhere Kalorien-, Fett- und Natriumzufuhr mit ungünstigen Gesundheitswerten wie erhöhtem BMI und systolischem Blutdruck einhergehen. Zucker weist hingegen keine klare Verbindung zu diesen Werten auf. Auch der Zusammenhang zwischen Kalorienaufnahme und BMI ist nur schwach – viele Einflussfaktoren wie Bewegung oder Stoffwechsel spielen hier mit hinein.

Insgesamt unterstreichen die Ergebnisse die Bedeutung ausgewogener Ernährung – insbesondere im Hinblick auf Energie- und Salzzufuhr – für die Prävention chronischer Erkrankungen. Gleichzeitig wird deutlich, dass individuelle Unterschiede berücksichtigt werden müssen und einfache Empfehlungen oft nicht ausreichen.


<img src="https://github.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/raw/96c605d37282bef8f5b6200a3585b575c9a6b2ce/Semesterarbeit/Images/Zusammenfassung_%C3%9Cberblick.png" alt="Beispielbild" width="500" />



---



**KAQ 1: In welchen Ernährungsmustern liegt eine potenziell unausgewogene Nährstoffzufuhr vor?**

Wir definieren **unausgewogen** als:

--> Überschreitung von WHO-Grenzwerten für:

- Fett (> 75 g/Tag)

- Natrium (> 2000 mg/Tag)

- Kalorien (> 2500 kcal/Tag)

In [ ]:
# 1. Datensätze laden und vorbereiten
# Was passiert hier?
# Wir wählen nur relevante Spalten und verbinden alle Daten über die Teilnehmer-ID SEQN.

# 1. CSV-Dateien laden
diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv")
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

diet_cols = ['SEQN', 'DR1TKCAL', 'DR1TTFAT', 'DR1TSODI']
examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']
questionnaire_cols = ['SEQN', 'DIQ010']

diet_small = diet[diet_cols]
examination_small = examination[examination_cols]
questionnaire_small = questionnaire[questionnaire_cols]

# Merge & Clean
df = diet_small.merge(examination_small, on="SEQN", how="inner")
df = df.merge(questionnaire_small, on="SEQN", how="inner")
df_clean = df.dropna().copy()

In [ ]:
# 2. Gesundheitsstatus zuweisen
# Was passiert hier?
# Wir klassifizieren Personen als:
# - Diabetiker, Bluthochdruck, Übergewicht oder "Keine der drei Diagnosen"

def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Keine der drei Diagnosen'

df_clean['Gesundheitsstatus'] = df_clean.apply(classify_health, axis=1)

In [ ]:
# 3. Schwellenwert-Spalten erstellen (WHO-Kriterien)
# Was passiert hier?
# Wir markieren jede Person mit True/False, je nachdem ob sie den Grenzwert überschreitet.

df_clean['zu_viel_Fett'] = df_clean['DR1TTFAT'] > 75
df_clean['zu_viel_Natrium'] = df_clean['DR1TSODI'] > 2000
df_clean['zu_viel_Kalorien'] = df_clean['DR1TKCAL'] > 2500

In [ ]:
# 4. Prozentuale Überschreitungen je Gesundheitsgruppe berechnen
# Was passiert hier?
# Wir gruppieren nach Gesundheitsstatus  und berechnen den Anteil (%) der Personen, die den Grenzwert überschreiten.

fett_risiko = df_clean.groupby('Gesundheitsstatus')['zu_viel_Fett'].mean() * 100
natrium_risiko = df_clean.groupby('Gesundheitsstatus')['zu_viel_Natrium'].mean() * 100
kalorien_risiko = df_clean.groupby('Gesundheitsstatus')['zu_viel_Kalorien'].mean() * 100

# Zusammenfassen in Tabelle
schwellenwerte = pd.DataFrame({
    'Über Fettgrenze (%)': fett_risiko.round(1),
    'Über Natriumgrenze (%)': natrium_risiko.round(1),
    'Über Kaloriengrenze (%)': kalorien_risiko.round(1)
})

In [ ]:
# 5. Visualisierung: Gruppierter Balkenplot
# Was passiert hier?
# Wir zeigen die Anteile je Gruppe grafisch, um Muster sofort zu erkennen.

import matplotlib.pyplot as plt
schwellenwerte.plot(kind="bar", figsize=(10, 6))
plt.title("Anteil der Personen über WHO-Grenzwerte nach Gesundheitsstatus")
plt.ylabel("Anteil (%)")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.legend(title="Nährstoffgrenze")
plt.tight_layout()
plt.show()

**Ergebnis:**

Man erkennt visuell und rechnerisch:

- **Übergewichtige und Bluthochdruckpatienten** haben den höchsten Anteil an überhöhter Fett- und Kalorienzufuhr.

- **Natriumaufnahme** ist in allen Gruppen sehr hoch.

- **Diabetiker** zeigen leicht bessere Werte, aber immer noch beachtliche Grenzwertüberschreitungen.

##Einleitung zur Visualisierungsanalyse (KAQ1)

Die folgenden Visualisierungen dienen der explorativen Analyse von Ernährungsmustern im Zusammenhang mit häufigen chronischen Erkrankungen: **Übergewicht**, **Bluthochdruck** und **Diabetes**. Ziel ist es, zu identifizieren, in welchen Gruppen eine potenziell **unausgewogene Nährstoffzufuhr** vorliegt und wie sich diese in Bezug auf zentrale Nährstoffe wie **Fett**, **Natrium (Salz)** und **Kalorien** äußert.

Als Referenz dienen dabei die Ernährungsempfehlungen der **Weltgesundheitsorganisation (WHO)**, insbesondere in Bezug auf Grenzwerte für die tägliche Zufuhr:
- Fett: ≤ 75 g
- Natrium: ≤ 2000 mg
- Energie: je nach Bedarf, grob ≤ 2500 kcal

Die Visualisierungen kombinieren **gruppenbasierte Auswertungen** (z. B. Boxplots nach Gesundheitsstatus) mit **verteilungsorientierten Darstellungen** (Histogramme) sowie einer **Beziehungsgrafik** (Scatterplot), um sowohl individuelle Muster als auch statistisch auffällige Tendenzen sichtbar zu machen.

Durch diese Analyse werden erste Hypothesen über Zusammenhänge zwischen **Ernährungsverhalten und gesundheitlichem Risiko** unterstützt. Die Ergebnisse liefern wertvolle Hinweise für Prävention, Gesundheitsförderung und individuelle Interventionen.

**1. Boxplot: Fettaufnahme nach Gesundheitsstatus**

Diese Visualisierung zeigt, wie stark die Fettzufuhr zwischen den verschiedenen Gesundheitsgruppen variiert.

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(8, 5))
sns.boxplot(data=df_clean, x='Gesundheitsstatus', y='DR1TTFAT')
plt.title("Fettaufnahme (g) nach Gesundheitsstatus")
plt.ylabel("Fettaufnahme (g)")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Erklärung:**

Diese Visualisierung zeigt die Verteilung der täglichen Fettaufnahme in den verschiedenen Gesundheitsgruppen. Auffällig ist, dass Personen mit Übergewicht und Bluthochdruck im Mittel höhere Fettmengen konsumieren als Personen ohne diese Diagnosen. Dies deutet auf ein potenziell unausgewogenes Ernährungsmuster mit überhöhter Fettzufuhr in diesen Gruppen hin. Gleichzeitig zeigt die starke Streuung innerhalb der Gruppen, dass auch individuelle Unterschiede eine Rolle spielen.

**2. Boxplot: Natriumaufnahme nach Gesundheitsstatus**

Die folgende Darstellung veranschaulicht die tägliche Salzzufuhr in den einzelnen Gesundheitsgruppen.

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_clean, x='Gesundheitsstatus', y='DR1TSODI')
plt.title("Natriumaufnahme (mg) nach Gesundheitsstatus")
plt.ylabel("Natriumaufnahme (mg)")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Erklärung:**

Diese Darstellung macht deutlich, dass die Natriumaufnahme in allen Gruppen hoch ist. Der Großteil der Personen überschreitet den von der WHO empfohlenen Grenzwert von 2000 mg Natrium pro Tag – unabhängig vom Gesundheitsstatus. Besonders für Personen mit Bluthochdruck ist dies kritisch, da ein hoher Salzkonsum als Risikofaktor für Bluthochdruck gilt. Diese Visualisierung unterstützt die Aussage, dass übermäßiger Salzkonsum ein weit verbreitetes Problem ist.

**3. Boxplot: Kalorienaufnahme nach Gesundheitsstatus**

Dieser Plot zeigt die Unterschiede in der Energieaufnahme zwischen den Gesundheitszuständen.

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_clean, x='Gesundheitsstatus', y='DR1TKCAL')
plt.title("Kalorienaufnahme (kcal) nach Gesundheitsstatus")
plt.ylabel("Kalorienaufnahme (kcal)")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Erklärung:**

Der Boxplot vergleicht die Kalorienaufnahme zwischen den Gesundheitsgruppen. Übergewichtige Personen zeigen tendenziell eine höhere mittlere Kalorienzufuhr im Vergleich zu den anderen Gruppen, was auf ein unausgewogenes Ernährungsmuster mit überhöhter Energieaufnahme hindeutet. Die Überschneidung der Boxen zeigt aber auch, dass es innerhalb jeder Gruppe Personen mit sehr unterschiedlichem Kalorienverhalten gibt.

**4. Histogramm: Verteilung der Fettaufnahme (gesamt)**

Die folgende Histogramm-Darstellung gibt einen Überblick über die Verteilung der Fettzufuhr in der gesamten Stichprobe.

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df_clean['DR1TTFAT'], bins=30, kde=True)
plt.title("Verteilung der Fettaufnahme in der Gesamtbevölkerung")
plt.xlabel("Fettaufnahme (g)")
plt.tight_layout()
plt.show()

**Erklärung:**

Dieses Histogramm zeigt die generelle Verteilung der Fettaufnahme in der untersuchten Bevölkerung. Die Verteilung ist deutlich rechtsschief – das heißt: Während viele Personen moderate Fettmengen konsumieren, gibt es auch eine kleinere Gruppe mit extrem hoher Fettaufnahme. Dieses Muster legt nahe, dass einige Personen potenziell stark unausgewogene Ernährungsgewohnheiten aufweisen, auch wenn ihr Gesundheitsstatus (noch) nicht auffällig ist.

**5. Scatterplot: Natriumaufnahme vs. systolischer Blutdruck**

Diese Visualisierung untersucht den möglichen Zusammenhang zwischen Salzkonsum und erhöhtem Blutdruck.

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df_clean, x='DR1TSODI', y='BPXSY1', hue='Gesundheitsstatus')
plt.title("Zusammenhang: Natriumaufnahme vs. systolischer Blutdruck")
plt.xlabel("Natriumaufnahme (mg)")
plt.ylabel("Systolischer Blutdruck (mmHg)")
plt.tight_layout()
plt.show()

**Erklärung:**

In dieser Visualisierung wird der Zusammenhang zwischen Natriumaufnahme und systolischem Blutdruck gezeigt. Während kein klarer linearer Trend erkennbar ist, deutet die Streuung der Punkte darauf hin, dass hohe Natriumaufnahme bei manchen Personen mit erhöhtem Blutdruck einhergeht – insbesondere bei Personen mit Bluthochdruckdiagnose. Diese Darstellung unterstreicht, dass die gesundheitliche Wirkung von zu viel Salz individuell unterschiedlich ist, aber dennoch ein Risikofaktor bleibt.

<h3>Die Visualisierungen in Bezug zu KAQ1</h3>

<table>
  <thead>
    <tr>
      <th>Visualisierung</th>
      <th>Was sie zeigt</th>
      <th>Beitrag zur KAQ1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><b>Boxplot Fettaufnahme nach Gruppe</b></td>
      <td>Gruppen mit systematisch höherer Fettzufuhr (z. B. Übergewichtige)</td>
      <td>✅ zeigt "fettlastige" Muster</td>
    </tr>
    <tr>
      <td><b>Boxplot Natriumaufnahme nach Gruppe</b></td>
      <td>Belegt hohe Salzzufuhr in allen Gruppen – besonders relevant für Bluthochdruck</td>
      <td>✅ unausgewogenes Salzverhalten sichtbar</td>
    </tr>
    <tr>
      <td><b>Boxplot Kalorienaufnahme</b></td>
      <td>Zeigt: Übergewichtige konsumieren im Schnitt mehr Energie</td>
      <td>✅ übermäßige Energiezufuhr als Muster</td>
    </tr>
    <tr>
      <td><b>Histogramm Fettaufnahme (gesamt)</b></td>
      <td>Verteilung: viele nehmen sehr viel Fett auf → streuende, unausgewogene Verteilung</td>
      <td>✅ zeigt allgemeines Risiko in Bevölkerung</td>
    </tr>
    <tr>
      <td><b>Scatterplot Natrium vs. Blutdruck</b></td>
      <td>Streuung, aber Hinweise auf Zusammenhang zwischen Salzkonsum und Blutdruck</td>
      <td>✅ Zusammenhang zwischen Muster & Auswirkung</td>
    </tr>
  </tbody>
</table>


**Gesamtfazit:**

Diese Visualisierungen beantworten **KAQ1 auf mehreren Ebenen**:

- Sie identifizieren **Gruppen mit erhöhtem Risiko**

- Sie zeigen **welche Nährstoffe** problematisch sind (Fett, Natrium, Kalorien)

- Sie verdeutlichen **Verhaltensmuster** in der Bevölkerung

- Sie liefern visuelle Argumente für die Bewertung der Ernährung als **ausgewogen oder unausgewogen**

---




## Zusammenfassung zu KAQ1: In welchen Ernährungsmustern liegt eine potenziell unausgewogene Nährstoffzufuhr vor?

Die Analyse der NHANES-Daten zeigt deutlich, dass bestimmte Gesundheitsgruppen – insbesondere Personen mit **Übergewicht** und **Bluthochdruck** – häufiger eine **unausgewogene Ernährung** aufweisen. Diese äußert sich in einer überdurchschnittlich hohen **Fett- und Kalorienzufuhr**, die in vielen Fällen die von der WHO empfohlenen Grenzwerte überschreitet. Gleichzeitig ist die **Natriumaufnahme** in nahezu allen Gruppen problematisch hoch, was insbesondere für Personen mit Bluthochdruck ein gesundheitliches Risiko darstellt.

Auch unter den Personen ohne manifeste Diagnose („keine der drei Diagnosen“) finden sich auffällige Ernährungsmuster, etwa in Form überhöhter Fett- oder Salzaufnahme. Dies legt nahe, dass nicht alle Risiken bereits klinisch sichtbar sind, sondern präventiv erkannt werden sollten.

Die Visualisierungen (Boxplots, Histogramme, Scatterplots) unterstützen diese Erkenntnisse, indem sie Unterschiede zwischen Gruppen sowie die Verteilung und Ausreißer bei Nährstoffzufuhren deutlich machen. Insgesamt zeigt sich, dass **bestimmte Ernährungsmuster – insbesondere hoher Energie- und Salzkonsum – mit einem erhöhten Gesundheitsrisiko einhergehen**, und damit als unausgewogen im Sinne dieser Fragestellung gelten.


<img src="https://github.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/raw/7bc2688a737ca1d1ddc5a8b40061fcf3a2f13b23/Semesterarbeit/Images/unausgewogene_Ern%C3%A4hrungsmuster.png" alt="Beispielbild" width="500" />



---



**KAQ 2: Welche Ernährungsmuster sind statistisch signifikant mit bestimmten Krankheitsbildern?**

**Vorgehensweise – aufbauend auf KAQ 1**

**In KAQ 1 hat man herausgefunden:**

- Übergewichtige & Bluthochdruck-Patienten haben häufiger:

  - hohe **Fettaufnahme**

  - hohe **Kalorienaufnahme**

  - hohe **Natriumaufnahme**

- Alle Gruppen zeigen zum Teil **problematische Muster** (auch „Gesunde“)

Mit KAQ 2 wird geprüft:

- **Sind diese Unterschiede zufällig – oder statistisch bedeutsam?**

- Gibt es **signifikante Zusammenhänge** (z. B. Fettaufnahme ↔ Übergewicht)?

**1. Kalorienaufnahme: Übergewicht vs. Gesund**

In [ ]:
# Stil für Seaborn
sns.set(style="whitegrid")

# 1. Sicherstellen, dass Spaltennamen korrekt sind
df_clean.columns = df_clean.columns.str.strip()

# 2. Subset für Übergewicht vs. Gesunde Personen mit gültigen Kalorienwerten
subset_kcal = df_clean[
    (df_clean['Gesundheitsstatus'].isin(['Übergewicht', 'Keine der drei Diagnosen'])) &
    (df_clean['DR1TKCAL'].notna())
]

# 3. Boxplot mit hue und Palette (ohne FutureWarning)
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=subset_kcal,
    x='Gesundheitsstatus',
    y='DR1TKCAL',
    hue='Gesundheitsstatus',
    dodge=False,
    legend=False,
    palette='Set2'
)

# 4. Titel & Achsenbeschriftung
plt.title("Kalorienaufnahme: Übergewicht vs. Gesund")
plt.ylabel("Kalorienaufnahme (kcal)")
plt.xlabel("Gesundheitsstatus")

# 5. p-Wert anzeigen
y_max = subset_kcal['DR1TKCAL'].max()
plt.text(0.5, y_max * 0.95, "p = 0.0496", ha='center', fontsize=11)

plt.tight_layout()
plt.show()



###Erklärung zur Visualisierung: Kalorienaufnahme bei Übergewicht vs. Gesund

Die Boxplot-Grafik zeigt die tägliche **Kalorienaufnahme (in Kilokalorien)** von übergewichtigen und gesunden Personen.

####Was ist zu sehen?
- Übergewichtige Personen nehmen im Schnitt **mehr Kalorien zu sich** als gesunde Personen.
- Der Unterschied ist **statistisch signifikant (p = 0.0496)** – wenn auch knapp.
- Die Werte streuen deutlich, was auf individuelle Unterschiede im Essverhalten hindeutet.

####Interpretation:
- Der signifikante Unterschied legt nahe, dass **eine erhöhte Kalorienzufuhr mit Übergewicht** zusammenhängt.  
- Dies steht im Einklang mit gängigen Erkenntnissen zur Energiebilanz: Wird über längere Zeit mehr Energie aufgenommen als verbraucht, steigt das Körpergewicht.  
- Dennoch zeigt die große Streuung, dass auch andere Faktoren wie körperliche Aktivität, Stoffwechselrate oder Ernährungsmuster relevant sind.

####Fazit:
Die Visualisierung unterstreicht, dass **eine hohe Kalorienzufuhr ein Schlüsselfaktor bei Übergewicht** ist – wenn auch nicht der einzige. Eine ausgewogene Energieaufnahme bleibt entscheidend für ein gesundes Körpergewicht.


**2. Fettaufnahme: Übergewicht vs. Gesund**

In [ ]:
# Stil festlegen
sns.set(style="whitegrid")

# Spaltennamen aufräumen (zur Sicherheit)
df_clean.columns = df_clean.columns.str.strip()

# Subset mit gültigen Fettwerten
subset_fett = df_clean[
    (df_clean['Gesundheitsstatus'].isin(['Übergewicht', 'Keine der drei Diagnosen'])) &
    (df_clean['DR1TTFAT'].notna())
]

# Plot erstellen mit hue zur Farbanpassung
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=subset_fett,
    x='Gesundheitsstatus',
    y='DR1TTFAT',
    hue='Gesundheitsstatus',
    dodge=False,
    legend=False,
    palette='Set2'
)

plt.title("Fettaufnahme: Übergewicht vs. Gesund")
plt.ylabel("Fettaufnahme (g)")
plt.xlabel("Gesundheitsstatus")

# p-Wert-Text einfügen
y_max = subset_fett['DR1TTFAT'].max()
plt.text(0.5, y_max * 0.95, "p = 0.0053", ha='center', fontsize=11)

plt.tight_layout()
plt.show()



### Erklärung zur Visualisierung: Fettaufnahme bei Übergewicht vs. Gesund

Die Boxplot-Grafik zeigt die Verteilung der täglichen **Fettaufnahme (in Gramm)** bei übergewichtigen Personen im Vergleich zu gesunden Teilnehmern („Keine der drei Diagnosen“).

#### Was ist zu sehen?
- Übergewichtige konsumieren im Mittel **deutlich mehr Fett** als gesunde Personen.
- Der Unterschied ist **statistisch signifikant (p = 0.0053)**.
- Die Streuung ist in beiden Gruppen vorhanden, bei Übergewichtigen aber tendenziell breiter.

#### Interpretation:
- Der signifikante Unterschied weist darauf hin, dass ein **hoher Fettkonsum mit Übergewicht assoziiert** ist.  
- Dies unterstützt ernährungswissenschaftliche Erkenntnisse, wonach übermäßige Fettzufuhr – vor allem bei gleichzeitig hohem Kalorienverbrauch – zur Gewichtszunahme beiträgt.  
- Dennoch zeigen die Ausreißer, dass individuelle Faktoren wie Bewegung oder Stoffwechsel eine Rolle spielen.

#### Fazit:
Die Analyse bestätigt, dass **eine überdurchschnittliche Fettaufnahme ein Risikomuster für Übergewicht** darstellt. Eine fettbewusste Ernährung kann daher zur Prävention beitragen.


**3. Natriumaufnahme: Bluthochdruck vs. Gesund**

In [ ]:
sns.set(style="whitegrid")

# Spaltennamen bereinigen (nur einmal im Notebook nötig)
df_clean.columns = df_clean.columns.str.strip()

# Subset mit gültigen Natriumwerten
subset_natrium = df_clean[
    (df_clean['Gesundheitsstatus'].isin(['Bluthochdruck', 'Keine der drei Diagnosen'])) &
    (df_clean['DR1TSODI'].notna())
]

# Boxplot mit korrektem hue-Handling
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=subset_natrium,
    x='Gesundheitsstatus',
    y='DR1TSODI',
    hue='Gesundheitsstatus',
    dodge=False,
    legend=False,
    palette='Set2'
)

plt.title("Natriumaufnahme: Bluthochdruck vs. Gesund")
plt.ylabel("Natriumaufnahme (mg)")
plt.xlabel("Gesundheitsstatus")

# p-Wert-Text
y_max = subset_natrium['DR1TSODI'].max()
plt.text(0.5, y_max * 0.95, "p = 0.2186", ha='center', fontsize=11)

plt.tight_layout()
plt.show()

### Erklärung zur Visualisierung: Natriumaufnahme bei Bluthochdruck vs. Gesund

Die Boxplot-Grafik zeigt die Verteilung der täglichen **Natriumaufnahme (in Milligramm)** bei Personen mit Bluthochdruck im Vergleich zu gesunden Teilnehmern („Keine der drei Diagnosen“).

#### Was ist zu sehen?
- Die mittlere Natriumaufnahme unterscheidet sich **nur geringfügig** zwischen den Gruppen.
- Der Unterschied ist **nicht signifikant (p = 0.2186)** – er könnte also zufällig entstanden sein.
- Die Werte streuen in beiden Gruppen stark, was auf große **individuelle Unterschiede** im Salzkonsum hindeutet.

#### Interpretation:
- Obwohl übermäßiger Salzkonsum ein bekannter Risikofaktor für Bluthochdruck ist, zeigt sich hier **kein signifikanter Unterschied**.  
- Mögliche Gründe könnten sein:
  - Viele Betroffene haben ihren **Salzkonsum bereits reduziert** (z. B. durch ärztliche Empfehlungen).
  - Die **Messung der Natriumzufuhr** basiert auf Selbstauskunft und ist ungenau.
  - **Weitere Risikofaktoren** wie Alter, Vererbung oder Medikation könnten den Blutdruck stärker beeinflussen.

#### Fazit:
In dieser Analyse zeigt sich **kein signifikanter Zusammenhang zwischen Natriumaufnahme und Bluthochdruck**. Das bedeutet jedoch nicht, dass Salzkonsum irrelevant ist – vielmehr sind **weitere Kontextfaktoren** entscheidend für die Interpretation.


**4. Zuckeraufnahme: Diabetes vs. Gesund**

In [ ]:
# 1. CSV-Dateien neu laden
diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv")
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

# 2. Relevante Spalten auswählen
diet_cols = ['SEQN', 'DR1TKCAL', 'DR1TSUGR', 'DR1TTFAT', 'DR1TSODI']
examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']
questionnaire_cols = ['SEQN', 'DIQ010']

# 3. Daten filtern
diet_small = diet[diet_cols]
examination_small = examination[examination_cols]
questionnaire_small = questionnaire[questionnaire_cols]

# 4. Daten zusammenführen
df = diet_small.merge(examination_small, on="SEQN", how="inner")
df = df.merge(questionnaire_small, on="SEQN", how="inner")

# 5. Fehlende Werte entfernen
df_clean = df.dropna().copy()

# 6. Gesundheitsstatus definieren
def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Keine der drei Diagnosen'

df_clean['Gesundheitsstatus'] = df_clean.apply(classify_health, axis=1)

# 7. Visualisierung vorbereiten
sns.set(style="whitegrid")

subset_diabetes = df_clean[
    (df_clean['Gesundheitsstatus'].isin(['Diabetes', 'Keine der drei Diagnosen'])) &
    (df_clean['DR1TSUGR'].notna())
].copy()

# 8. Boxplot erstellen
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=subset_diabetes,
    x='Gesundheitsstatus',
    y='DR1TSUGR',
    hue='Gesundheitsstatus',
    dodge=False,
    palette='Set2'
)

plt.title("Zuckeraufnahme: Diabetes vs. Gesund")
plt.ylabel("Zuckeraufnahme (g)")
plt.xlabel("Gesundheitsstatus")

# 9. p-Wert anzeigen
y_max = subset_diabetes['DR1TSUGR'].max()
plt.text(0.5, y_max * 0.95, "p = 0.0000", ha='center', fontsize=11)

# 10. Legende sicher entfernen – ohne Warnung
if plt.gca().get_legend():
    plt.gca().get_legend().remove()

plt.tight_layout()
plt.show()

### Erklärung zur Visualisierung: Zuckeraufnahme bei Diabetes vs. Gesund

Die Boxplot-Grafik zeigt die Verteilung der täglichen **Zuckeraufnahme (in Gramm)** bei Personen mit Diabetes im Vergleich zu gesunden Teilnehmern („Keine der drei Diagnosen“).

#### Was ist zu sehen?
- Diabetiker konsumieren im Mittel **weniger Zucker** als gesunde Personen.
- Der Unterschied ist **statistisch hochsignifikant (p = 0.0000)** – er ist sehr wahrscheinlich **nicht zufällig**, sondern systematisch vorhanden.
- Die Streuung ist in beiden Gruppen deutlich, bei Diabetikern jedoch mit einer **tendenziell geringeren Obergrenze**.

#### Interpretation:
- Die Ergebnisse deuten darauf hin, dass viele Diabetiker ihre Ernährung **nach der Diagnose bewusst anpassen** und den **Zuckerkonsum reduzieren** – etwa durch ärztliche Empfehlung oder gesteigertes Gesundheitsbewusstsein.  
- Der signifikante Unterschied lässt sich daher eher als **Folge der Erkrankung** interpretieren, **nicht als deren Ursache**.

#### Fazit:
Die Visualisierung zeigt exemplarisch, wie ein **signifikanter Unterschied** in den Daten sichtbar wird – ohne automatisch auf eine Ursache-Wirkung-Beziehung schließen zu müssen. Sie betont zudem die Rolle von **Verhaltensänderungen nach einer Diagnose** in der Ernährungspraxis.


## Zusammenfassung – KAQ 2: Welche Ernährungsmuster sind statistisch signifikant mit bestimmten Krankheitsbildern assoziiert?

Zur Beantwortung von KAQ 2 wurden gezielt **statistische Gruppenvergleiche in Form von t-Tests** durchgeführt. Ziel war es herauszufinden, ob sich die **Nährstoffaufnahme signifikant zwischen gesunden Personen und solchen mit chronischen Erkrankungen** wie Diabetes, Übergewicht oder Bluthochdruck unterscheidet. Dabei wurden vier zentrale Ernährungsmuster betrachtet: die Aufnahme von **Kalorien, Fett, Zucker und Natrium**.

Die Analyse zeigt, dass **übergewichtige Personen im Durchschnitt signifikant mehr Kalorien konsumieren** als gesunde Vergleichspersonen (p = 0.0496), was auf einen Zusammenhang zwischen erhöhter Energiezufuhr und Übergewicht hinweist. Noch deutlicher wird dieser Zusammenhang bei der Fettaufnahme: Hier ist der Unterschied mit einem p-Wert von 0.0053 nicht nur signifikant, sondern hoch signifikant. Das unterstreicht die Rolle fettreicher Ernährungsmuster bei der Entstehung von Übergewicht.

Im Fall von Diabetes zeigt sich ein umgekehrtes Bild: **Diabetiker nehmen signifikant weniger Zucker zu sich** als gesunde Personen (p = 0.0000). Dieser Unterschied lässt sich vermutlich auf eine bewusste Anpassung der Ernährung nach der Diagnose zurückführen – etwa durch ärztliche Empfehlungen oder ein gestiegenes Gesundheitsbewusstsein. Anders verhält es sich bei Bluthochdruck: Trotz des bekannten Zusammenhangs zwischen Salzkonsum und erhöhtem Blutdruck konnte **kein signifikanter Unterschied in der Natriumaufnahme** festgestellt werden (p = 0.2186). Dies weist auf komplexere Ursachen hin, beispielsweise individuelle Unterschiede, bereits erfolgte Diätanpassungen oder methodische Ungenauigkeiten in der Datenerhebung.

Zusammenfassend lässt sich sagen, dass **unausgewogene Ernährungsmuster – insbesondere eine hohe Kalorien- und Fettzufuhr – klar mit Übergewicht assoziiert sind**. Ebenso lässt sich eine deutliche Zuckerreduktion bei Diabetikern feststellen, vermutlich als Folge therapeutischer Maßnahmen. Für Bluthochdruck hingegen konnten keine signifikanten Unterschiede in der Natriumaufnahme nachgewiesen werden, was auf die Relevanz weiterer Einflussgrößen hinweist und verdeutlicht, dass nicht jeder Zusammenhang in Querschnittsdaten statistisch sichtbar wird.

<img src="https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/fab1725d8a7206ec0edb297092b41d006447fe0c/Semesterarbeit/Images/Signifikante_Ern%C3%A4hrungsmuster.png" alt="Beispielbild" width="500" />





---



**KAQ 3: Wie stark weichen die identifizierten Ernährungsmuster (Cluster/Risikogruppen) von den offiziellen Nährstoffempfehlungen (z. B. BLV) ab – und welche Nährstoffe sind dabei besonders kritisch in Bezug auf präventive Gesundheit?**

**Vorgehensweise – aufbauend auf KAQ 2**

**In KAQ 2 wurde analysiert:**

- Übergewichtige Personen konsumieren **signifikant mehr Kalorien und Fett**
- Diabetiker nehmen **signifikant weniger Zucker** zu sich
- Bei Bluthochdruck-Patienten ist **kein signifikanter Unterschied in der Natriumaufnahme** erkennbar
- Gesunde Personen zeigen ebenfalls **zum Teil kritische Ernährungsmuster**

Mit **KAQ 3** wird geprüft:

- **Wie stark weichen einzelne Gruppen (z. B. Übergewichtige, Diabetiker, Gesunde) von den offiziellen Ernährungsempfehlungen ab?**
- **Welche Nährstoffe sind besonders oft problematisch (zu hoch oder zu niedrig)?**
- Gibt es **bestimmte Risikogruppen oder Cluster**, die mehrere Empfehlungen gleichzeitig überschreiten?

Dazu wird analysiert:

- Die individuelle Nährstoffaufnahme mit **offiziellen Richtwerten** (z. B. BLV, WHO, D-A-CH)
- Die **Häufigkeit von Überschreitungen** in jeder Gruppe  
  _(z. B. „Wie viele Übergewichtige konsumieren zu viel Fett?“)_

--> **Ziel**: Kritische Abweichungen **sichtbar machen**, bewerten und daraus **präventive Handlungsschwerpunkte ableiten**  
_(z. B. Fettreduktion bei Übergewicht, Zuckerkontrolle bei Diabetikern)_


### Übersicht: Offizielle Nährstoffempfehlungen (pro Tag, Erwachsene)

| Nährstoff        | Empfehlung                             | Quelle             |
|------------------|-----------------------------------------|--------------------|
| **Kalorien**     | 2000–2500 kcal                         | D-A-CH-Referenzwerte / WHO |
| **Fett**         | max. 30 % der Energie (~70–80 g)       | WHO / BLV          |
| **Zucker**       | max. 50 g (≤ 10 % der Energiezufuhr)   | WHO / BLV          |
| **Natrium**      | max. 2000 mg (entspricht 5 g Salz)     | WHO / BLV          |

Hinweis: Die Werte sind Richtgrössen für gesunde Erwachsene mit durchschnittlichem Energiebedarf. Für Kinder, Schwangere oder Kranke gelten abweichende Empfehlungen.


In [ ]:

# 📌 1. CSV-Dateien einlesen
diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv")
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

# 📌 2. Relevante Spalten auswählen
diet_cols = ['SEQN', 'DR1TKCAL', 'DR1TSUGR', 'DR1TTFAT', 'DR1TSODI']
examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']
questionnaire_cols = ['SEQN', 'DIQ010']

diet_small = diet[diet_cols]
examination_small = examination[examination_cols]
questionnaire_small = questionnaire[questionnaire_cols]

# 📌 3. Daten zusammenführen
df = diet_small.merge(examination_small, on="SEQN", how="inner")
df = df.merge(questionnaire_small, on="SEQN", how="inner")
df_clean = df.dropna().copy()

# 📌 4. Gesundheitsstatus berechnen
def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Keine der drei Diagnosen'

df_clean['Gesundheitsstatus'] = df_clean.apply(classify_health, axis=1)
df_clean.columns = df_clean.columns.str.strip()

# 📌 5. Empfehlungen & Grenzwertüberschreitungen berechnen
kalorien_max = 2500
fett_max = 75
zucker_max = 50
natrium_max = 2000

df_clean['zu_viel_Kalorien'] = df_clean['DR1TKCAL'] > kalorien_max
df_clean['zu_viel_Fett'] = df_clean['DR1TTFAT'] > fett_max
df_clean['zu_viel_Zucker'] = df_clean['DR1TSUGR'] > zucker_max
df_clean['zu_viel_Natrium'] = df_clean['DR1TSODI'] > natrium_max

# 📌 6. Gruppierung nach Gesundheitsstatus (Anteil in %)
abweichungen_pro_gruppe = df_clean.groupby('Gesundheitsstatus')[
    ['zu_viel_Kalorien', 'zu_viel_Fett', 'zu_viel_Zucker', 'zu_viel_Natrium']
].mean().round(3) * 100

# 📌 7. Anzeige der Ergebnisse (Jupyter-kompatibel)
print("Grenzwertüberschreitungen nach Gesundheitsstatus (%)")
display(abweichungen_pro_gruppe)

### Erklärung der Tabelle: Grenzwertüberschreitungen nach Gesundheitsstatus (%)

Die Tabelle zeigt für jede Gesundheitsgruppe (z. B. Übergewichtige, Diabetiker, Gesunde), wie viele Personen den jeweiligen **offiziellen Grenzwert für Kalorien, Fett, Zucker oder Natrium** überschreiten.  
Die Werte sind als **Prozentanteile** angegeben – z. B. bedeutet „48 % bei Fett“, dass fast die Hälfte der Personen dieser Gruppe mehr Fett konsumiert als empfohlen.

#### Was die Spalten bedeuten:

| Spalte               | Bedeutung                                                           |
|----------------------|---------------------------------------------------------------------|
| **zu_viel_Kalorien** | Anteil der Personen mit täglicher Kalorienzufuhr über 2500 kcal     |
| **zu_viel_Fett**     | Anteil mit Fettzufuhr über 75 g/Tag (basierend auf WHO-Empfehlung)  |
| **zu_viel_Zucker**   | Anteil mit Zuckerzufuhr über 50 g/Tag (WHO: max. 10 % der Energie)  |
| **zu_viel_Natrium**  | Anteil mit Natriumzufuhr über 2000 mg/Tag (~5 g Salz)               |

#### Interpretation:

- **Zucker** wird von einem sehr hohen Anteil überschritten – sogar in der Gruppe „Keine der drei Diagnosen“
- **Fett** wird vor allem bei Übergewicht häufig überschritten – das passt zur Annahme eines Zusammenhangs
- **Natrium** liegt bei fast allen Gruppen über dem Grenzwert – ein verbreitetes Problem
- **Kalorien** werden besonders bei Übergewicht häufig überschritten, aber auch bei anderen Gruppen auffällig

#### Fazit:

Diese Übersicht zeigt, **welche Ernährungsmuster je nach Gesundheitsstatus besonders kritisch sind**.  
Sie macht sichtbar, wo **gezielte Präventionsmaßnahmen** (z. B. Reduktion von Fett oder Salz) nötig wären, und liefert eine evidenzbasierte Grundlage zur Beantwortung von **KAQ 3**.


### Was lässt sich mit dieser Liste analysieren?

Die Liste zeigt, **wie häufig einzelne Gesundheitsgruppen** (z. B. Übergewichtige, Diabetiker) bestimmte **empfohlene Nährstoffgrenzen überschreiten**. Daraus lassen sich mehrere wichtige Analysen ableiten:

---

#### 1. **Welche Gruppen sind besonders betroffen?**
Analysiere, welcher Gesundheitsstatus (z. B. „Übergewicht“) besonders häufig zu viel Kalorien, Fett, Zucker oder Natrium konsumiert.

---

#### 2. **Welche Nährstoffe sind generell kritisch?**
Erkenne, welche Nährstoffe **unabhängig vom Gesundheitsstatus** besonders oft über dem Richtwert liegen – also potenziell **bevölkerungsweit problematisch** sind.

---

#### 3. **Risikomuster identifizieren**
Identifiziere kritische Kombinationen – z. B. Gruppen mit gleichzeitig hoher Fett- und Kalorienaufnahme.  
Das ist besonders relevant für gezielte Prävention.

---

#### 4. **Begründete Präventionsempfehlungen ableiten**
Nutze die Liste als datenbasierte Grundlage für Ernährungsempfehlungen, z. B.:

- **Salzkonsum senken** (betrifft fast alle Gruppen)
- **Fettreduktion gezielt bei Übergewichtigen**
- **Zuckerkonsum auch bei scheinbar Gesunden kritisch betrachten**

---

Die Liste hilft, sowohl **individuelle Risikoprofile** als auch **gesellschaftlich relevante Ernährungsmuster** zu erkennen – und liefert damit eine fundierte Grundlage zur Beantwortung von **KAQ 3**.


**1. Welche Gruppen sind besonders betroffen?**

**Analyse:**

Zeigt für jede Gesundheitsgruppe (z. B. Übergewicht, Diabetes), wie häufig die jeweiligen Nährstoffgrenzwerte überschritten werden.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
abweichungen_pro_gruppe.plot(kind='bar', ax=plt.gca())
plt.title("Anteil der Personen über Grenzwerte – nach Gesundheitsstatus")
plt.ylabel("Anteil in %")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.legend(title="Nährstoff")
plt.tight_layout()
plt.show()


**Interpretation:**

Man erkennt, welche Gruppen besonders auffallen:

→ z. B. Übergewichtige überschreiten häufiger Fett und Kalorien.

→ Auch „Gesunde“ haben auffällig hohe Zucker- und Salzwerte.

**2. Welche Nährstoffe sind generell kritisch?**

**Analyse:**

Berechnet den Durchschnitt pro Nährstoff – unabhängig von der Gruppe. So erkennt man, welche Nährstoffe allgemein problematisch sind.

In [ ]:
kritische_naehrstoffe = abweichungen_pro_gruppe.mean().sort_values(ascending=False)

plt.figure(figsize=(6, 4))
kritische_naehrstoffe.plot(kind='barh', color='darkorange')
plt.title("Durchschnittliche Überschreitungen pro Nährstoff")
plt.xlabel("Anteil in %")
plt.tight_layout()
plt.show()


**Interpretation:**

**Zucker und Natrium** werden in allen Gruppen sehr häufig überschritten.

→ Diese Nährstoffe sind **gesamtgesellschaftlich kritisch** – unabhängig vom Gesundheitsstatus.

→ Hinweis auf verbreitete Fehlernährung.

**3. Risikomuster identifizieren**

**Analyse:**

Untersucht die gleichzeitige Überschreitung von Fett und Kalorien – ein typisches Muster bei unausgewogener Ernährung.

In [ ]:
kombiniert = df_clean.groupby('Gesundheitsstatus').apply(
    lambda g: ((g['zu_viel_Fett']) & (g['zu_viel_Kalorien'])).mean()
).round(3) * 100

kombiniert.plot(kind='bar', color='firebrick', figsize=(7, 4))
plt.title("Fett + Kalorien gleichzeitig überschritten")
plt.ylabel("Anteil in %")
plt.xlabel("Gesundheitsstatus")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Interpretation:**

Dieses Muster tritt am häufigsten bei **Übergewichtigen** auf.

→ Bestätigt die Hypothese, dass **energiereiche, fettreiche Ernährung ein Risiko für Übergewicht** darstellt.

→ Dient als Hinweis für **zielgerichtete Prävention**.

## Zusammenfassung - KAQ 3: Wie stark weichen die identifizierten Ernährungsmuster (Cluster/Risikogruppen) von den offiziellen Nährstoffempfehlungen (z. B. BLV) ab – und welche Nährstoffe sind dabei besonders kritisch in Bezug auf präventive Gesundheit?

Die Analyse der Grenzwertüberschreitungen nach Gesundheitsstatus liefert zentrale Erkenntnisse im Kontext von **KAQ 3**. Dabei wurde untersucht, inwieweit bestimmte Gesundheitsgruppen – wie Personen mit **Übergewicht**, **Bluthochdruck**, **Diabetes** oder **ohne Diagnose** – die empfohlenen Tagesgrenzwerte für **Kalorien**, **Fett**, **Zucker** und **Natrium** überschreiten.

Zunächst zeigte sich, dass insbesondere **Übergewichtige** deutlich häufiger die Grenzwerte für **Fett und Kalorien** überschreiten als andere Gruppen. Dies legt nahe, dass eine hohe Energie- und Fettzufuhr maßgeblich zur Entstehung von Übergewicht beitragen kann. Interessanterweise weisen auch Personen ohne Diagnose („gesunde Gruppe“) sehr hohe Überschreitungsraten bei **Zucker und Natrium** auf. Das deutet darauf hin, dass ungünstige Ernährungsmuster nicht ausschließlich bei erkrankten Personen auftreten, sondern auch in der **Gesamtbevölkerung** weit verbreitet sind.

In einem weiteren Schritt wurde analysiert, welche Nährstoffe **gruppenunabhängig** besonders kritisch sind. Dabei wurde ersichtlich, dass vor allem **Zucker** und **Natrium** über alle Gesundheitsgruppen hinweg am häufigsten über dem empfohlenen Richtwert liegen. Diese Erkenntnis ist wichtig, da sie auf ein strukturelles Problem in der Ernährung vieler Menschen hinweist – **unabhängig vom aktuellen Gesundheitsstatus**.

Ein dritter Fokus lag auf der **Identifikation typischer Risikomuster** – also Kombinationen von Grenzwertüberschreitungen. Hier zeigte sich, dass insbesondere Übergewichtige auffallend häufig sowohl **zu viel Fett als auch zu viele Kalorien** aufnehmen. Diese doppelte Belastung ist besonders gesundheitskritisch und kann als Ansatzpunkt für **gezielte präventive Maßnahmen** dienen.

Insgesamt zeigt die Analyse: Bestimmte Ernährungsmuster – insbesondere ein hoher Fett- und Energiegehalt – sind **signifikant mit dem Gesundheitsstatus verknüpft**. Gleichzeitig existieren auch in vermeintlich „gesunden“ Gruppen verbreitete Risiken, vor allem in Bezug auf Zucker- und Salzkonsum. Diese Ergebnisse unterstreichen die Bedeutung von **breitenwirksamer Ernährungserziehung** und **zielgerichteter Prävention** auf Basis wissenschaftlich fundierter Daten.

---



**KAQ 4: Wie können auf Basis individueller Ernährungsmuster und assoziierter Krankheitsrisiken personalisierte Ernährungsempfehlungen abgeleitet werden, um präventiv gegenzusteuern – und welche konkreten Lebensmittel(-gruppen) eignen sich zur Umsetzung?**

## Vorgehensweise zur Beantwortung von KAQ 4

**KAQ 4:**  
*Wie können auf Basis individueller Ernährungsmuster und assoziierter Krankheitsrisiken personalisierte Ernährungsempfehlungen abgeleitet werden, um präventiv gegenzusteuern – und welche konkreten Lebensmittel(-gruppen) eignen sich zur Umsetzung?*

---

### 1. Erkenntnisse aus KAQ 3 nutzen

Die Analyse aus KAQ 3 zeigt, welche Gesundheitsgruppen bestimmte Nährstoffgrenzwerte besonders häufig überschreiten:

| Risikogruppe       | Auffällige Ernährungsmuster          | Kritische Nährstoffe      |
|--------------------|--------------------------------------|----------------------------|
| Übergewicht        | Hohe Kalorien- und Fettzufuhr        | Energie, Fett              |
| Bluthochdruck      | Hohe Natriumzufuhr                   | Salz                       |
| Diabetiker         | Zuckerkonsum tendenziell angepasst   | Zucker (vor der Diagnose)  |
| Gesunde            | Hoher Anteil mit zu viel Zucker/Salz | Zucker, Natrium            |

---

### 2. Individualisierte Risikoprofile bilden

Auf Basis von Gesundheitsstatus + Nährstoffverhalten können Ernährungstypen erstellt werden:

| Ernährungstyp             | Risiko              | Ziel                          |
|---------------------------|---------------------|-------------------------------|
| „Kalorienreich & fettreich“ | Übergewicht         | Energiereduktion, Sättigung   |
| „Salzlastig“               | Bluthochdruck        | Natriumreduktion, Kaliumzufuhr|
| „Zuckeraffin“              | Prädiabetes          | Zuckerreduktion, Ballaststoffe|

---

### 3. Personalisierte Ernährungsempfehlungen ableiten

**a) Für Übergewichtige:**

- **Empfehlung:** Kalorien- und Fettzufuhr senken  
- **Lebensmittelgruppen:**  
  ✅ Gemüse, Hülsenfrüchte, Vollkorn  
  ❌ Frittierte Speisen, fettreiche Wurst, süße Snacks

**b) Für Personen mit Bluthochdruck:**

- **Empfehlung:** Salzkonsum senken, Kaliumzufuhr erhöhen  
- **Lebensmittelgruppen:**  
  ✅ Bananen, Blattgemüse, ungesalzene Nüsse  
  ❌ Fertigprodukte, Chips, salziges Brot

**c) Für Zuckerrisikogruppe (z. B. Prädiabetiker):**

- **Empfehlung:** Zuckerreduktion & ballaststoffreiche Kost  
- **Lebensmittelgruppen:**  
  ✅ Beeren, Haferflocken, Linsen  
  ❌ Limonaden, gesüßte Cerealien, Süßgebäck

---

### 4. Umsetzung in Tools (optional)

Ein Algorithmus kann basierend auf dem Nährstoffverhalten:

- Risikoprofile erkennen
- passende Ernährungsempfehlungen ableiten
- gezielte Lebensmittelvorschläge ausgeben

➡ (Optional: Umsetzung in Python möglich)

---

### Fazit:

Die Erkenntnisse aus KAQ 3 bilden eine fundierte Grundlage zur **personalisierbaren Prävention**.  
Individuelle Ernährungsmuster lassen sich mit gezielten Lebensmittelvorschlägen verbinden – für eine **alltagsnahe Umsetzung** und **nachhaltige Gesundheitsförderung**.


In [ ]:
# 1. Ernährungsmuster klassifizieren
def classify_pattern(row):
    if row['zu_viel_Kalorien'] and row['zu_viel_Fett']:
        return 'Fett- & Kalorienreich'
    elif row['zu_viel_Zucker']:
        return 'Zuckerreich'
    elif row['zu_viel_Natrium']:
        return 'Salzreich'
    else:
        return 'Unauffällig'

df_clean['Ernährungsmuster'] = df_clean.apply(classify_pattern, axis=1)

# 2. Personalisierte Ernährungsempfehlung generieren
def generate_recommendation(row):
    tips = []

    if row['Ernährungsmuster'] == 'Fett- & Kalorienreich':
        tips.append("Fett und Kalorien senken – mehr Gemüse, weniger Fast Food und Wurst")
    if row['Ernährungsmuster'] == 'Zuckerreich':
        tips.append("Zucker reduzieren – Süßgetränke und Süßigkeiten vermeiden")
    if row['Ernährungsmuster'] == 'Salzreich':
        tips.append("Salz reduzieren – Fertigprodukte meiden, kaliumreiche Kost wählen")
    if row['Ernährungsmuster'] == 'Unauffällig':
        tips.append("Muster unauffällig – gesundes Verhalten fortsetzen")

    return "; ".join(tips)

df_clean['Empfehlung'] = df_clean.apply(generate_recommendation, axis=1)

# 3. Ergebnis anzeigen
df_clean[['SEQN', 'Gesundheitsstatus', 'Ernährungsmuster', 'Empfehlung']].head()


### Automatisierte Klassifikation von Ernährungsmustern und Empfehlungen

Zur Beantwortung von KAQ 4 wurde ein regelbasierter Klassifikationsansatz implementiert, um individuelle Ernährungsmuster auf Basis objektiver Nährstoffdaten zu identifizieren und daraus personalisierte Empfehlungen abzuleiten.

Basierend auf den Nährstoffwerten (Kalorien, Fett, Zucker, Natrium) wurde geprüft, ob eine Person die offiziellen Richtwerte überschreitet. Daraus wurden vier Ernährungsmuster abgeleitet:

- **Fett- & Kalorienreich** (Kalorien > 2500 kcal & Fett > 75 g)
- **Zuckerreich** (Zucker > 50 g)
- **Salzreich** (Natrium > 2000 mg)
- **Unauffällig** (keine Grenzüberschreitung)

Für jedes dieser Muster wurde eine konkrete Ernährungsempfehlung generiert. Diese Empfehlungen zielen auf Prävention und Gesundheitsförderung ab und sind direkt aus den identifizierten Ernährungsverhalten abgeleitet.

Beispielsweise werden Personen mit einem fett- und kalorienreichen Muster dazu angeleitet, mehr pflanzliche Lebensmittel zu konsumieren und energiedichte Produkte zu reduzieren. Bei einem zuckerreichen Muster wird empfohlen, auf gezuckerte Getränke und Süßigkeiten zu verzichten. Auch für salzreiche Ernährungsmuster werden alltagsnahe Maßnahmen zur Reduktion von Kochsalz und Erhöhung der Kaliumzufuhr vorgeschlagen.

Die Methode erlaubt es, aus standardisierten Ernährungsdaten automatisiert individuelle Empfehlungen zu erzeugen – ein zentraler Schritt in Richtung personalisierter Ernährung und präventiver Gesundheitsförderung.


<h3>Entscheidungsbaum zur Klassifikation von Ernährungsmustern</h3>
<img src="https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/15f668521c4a305fb6a7f9f54e54969b743167aa/Semesterarbeit/Images/Entscheidungsbaum_Klassifikation_Ern%C3%A4hrungsmuster.png" width="50%">


**Visualisierungen zur Klassifkation der Ernährungsmuster**

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

# Feature-Auswahl: binäre Nährstoffüberschreitungen
features = ['zu_viel_Kalorien', 'zu_viel_Fett', 'zu_viel_Zucker', 'zu_viel_Natrium']
X = df_clean[features]
y = df_clean['Ernährungsmuster']

# Entscheidungsbaum trainieren
tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X, y)

# Visualisierung
plt.figure(figsize=(12, 6))
plot_tree(tree, feature_names=features, class_names=tree.classes_, filled=True, rounded=True)
plt.title("Visualisierung der Ernährungsmuster-Klassifikation als Entscheidungsbaum")
plt.show()



**Wie kann dieser Entscheidungsbaum erweitert werden?**

Nun geht es nicht mehr nur um Ernährungsmuster, sondern auch um die Frage:

„Wie können wir auf Basis dieser Muster konkrete Maßnahmen zur Prävention von Krankheiten ableiten?“


**1. Risikoscoring / Risikoprofil**

Berechne für jede Person einen **Risiko-Score**, z. B. von 0–100:

- 1 Punkt pro überschrittenem Nährstoff-Grenzwert

- Bonuspunkte für Kombinationen (z. B. Fett+Kalorien = 3)

**Mehrwert:** Man bekommt eine **graduelle Einschätzung** statt nur Klassen.

➕ Visualisierbar mit z. B. Heatmap oder Balkenplot

➕ Ideal zur Priorisierung von Interventionen

In [ ]:
# Optional: nur kleine Stichprobe laden
diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv").sample(1000, random_state=1)
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

# Spalten
diet_cols = ['SEQN', 'DR1TKCAL', 'DR1TTFAT', 'DR1TSUGR', 'DR1TSODI']
examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']
questionnaire_cols = ['SEQN', 'DIQ010']

diet = diet[diet_cols]
examination = examination[examination_cols]
questionnaire = questionnaire[questionnaire_cols]

# Merge & Clean
df = diet.merge(examination, on="SEQN").merge(questionnaire, on="SEQN")
df = df.dropna()

# Grenzwerte definieren
df['zu_viel_Kalorien'] = df['DR1TKCAL'] > 2500
df['zu_viel_Fett'] = df['DR1TTFAT'] > 75
df['zu_viel_Zucker'] = df['DR1TSUGR'] > 50
df['zu_viel_Natrium'] = df['DR1TSODI'] > 2000

# Risiko-Score berechnen
df['Risiko_Score'] = (
    df['zu_viel_Kalorien'].astype(int) +
    df['zu_viel_Fett'].astype(int) +
    df['zu_viel_Zucker'].astype(int) +
    df['zu_viel_Natrium'].astype(int) +
    ((df['zu_viel_Kalorien']) & (df['zu_viel_Fett'])).astype(int)  # Bonus
)

# Visualisierung
sns.set(style="whitegrid")
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Risiko_Score', palette='flare')
plt.title("Verteilung des Risiko-Scores")
plt.xlabel("Risiko-Score (0–5)")
plt.ylabel("Anzahl Personen")
plt.tight_layout()
plt.show()


**Interpretation:**

- Score von **0**: keine Grenzwerte verletzt → Ernährung vermutlich ausgewogen

- Score von **5**: alle Grenzwerte überschritten → höchste Risikogruppe

- Dazwischen: zunehmende Relevanz für Prävention und Beratung

**ML-Modell zur Krankheitsrisiko-Vorhersage mit Risiko-Score**

- Berechnung des Risiko-Scores,
- Erstellung der Zielvariable ("ist_krank"),
- Trainierung eines Klassifikationsmodells (Logistische Regression),
- Auswertung der Vorhersagequalität (Confusion Matrix & Report).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 1. CSV-Dateien laden
diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv")
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

# 2. Relevante Spalten auswählen
diet_cols = ['SEQN', 'DR1TKCAL', 'DR1TTFAT', 'DR1TSUGR', 'DR1TSODI']
examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']
questionnaire_cols = ['SEQN', 'DIQ010']

diet = diet[diet_cols]
examination = examination[examination_cols]
questionnaire = questionnaire[questionnaire_cols]

# 3. Merge & NaN entfernen
df = diet.merge(examination, on="SEQN").merge(questionnaire, on="SEQN")
df = df.dropna()

# 4. Gesundheitsstatus klassifizieren
def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Keine der drei Diagnosen'

df['Gesundheitsstatus'] = df.apply(classify_health, axis=1)

# 5. Schwellenwerte definieren
df['zu_viel_Kalorien'] = df['DR1TKCAL'] > 2500
df['zu_viel_Fett'] = df['DR1TTFAT'] > 75
df['zu_viel_Zucker'] = df['DR1TSUGR'] > 50
df['zu_viel_Natrium'] = df['DR1TSODI'] > 2000

# 6. Risiko-Score berechnen (inkl. Bonus für Kalorien+Fett)
df['Risiko_Score'] = (
    df['zu_viel_Kalorien'].astype(int) +
    df['zu_viel_Fett'].astype(int) +
    df['zu_viel_Zucker'].astype(int) +
    df['zu_viel_Natrium'].astype(int) +
    ((df['zu_viel_Kalorien']) & (df['zu_viel_Fett'])).astype(int)
)

# 7. Zielvariable: Hat die Person eine chronische Erkrankung?
df['ist_krank'] = df['Gesundheitsstatus'].isin(['Übergewicht', 'Bluthochdruck', 'Diabetes']).astype(int)

# 8. Train-Test-Split
X = df[['Risiko_Score']]
y = df['ist_krank']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 9. Modell trainieren
model = LogisticRegression()
model.fit(X_train, y_train)

# 10. Vorhersagen & Auswertung
y_pred = model.predict(X_test)

print("📊 Klassifikationsbericht:\n")
print(classification_report(y_test, y_pred, digits=3))

# 11. Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix: Risiko-Score → Krankheitsstatus")
plt.xlabel("Vorhergesagt")
plt.ylabel("Tatsächlich")
plt.tight_layout()
plt.show()


## Machine Learning: Ab wann beginnt der eigentliche Analyseprozess?

Für die Beantwortung von **KAQ 4 mittels Machine Learning** beginnt der relevante Teil der Analyse bei **Schritt 4** – also ab dem Moment, wo die Daten für das Modell in die richtige Form gebracht werden.

---

### Schritt 4: Risiko-Score berechnen

Hier wird das zentrale Eingabemerkmal (`Feature`) erzeugt:

- Der **Risiko-Score** ist ein Wert zwischen **0 und 5**
- Er basiert auf der Anzahl überschrittener Grenzwerte für Kalorien, Fett, Zucker und Salz
- Ein **Bonuspunkt** wird vergeben, wenn **Kalorien UND Fett** gleichzeitig zu hoch sind
- ➕ Der Score ist eine **kompakte Repräsentation eines Ernährungsmusters**

---

### Schritt 5: Zielvariable `ist_krank` definieren

Diese Variable gibt an, ob eine Person zu einer **Risikogruppe** gehört:

- `ist_krank = 1` → bei **Übergewicht**, **Bluthochdruck** oder **Diabetes**
- `ist_krank = 0` → bei **keiner der drei Diagnosen**
- ➕ Diese Zielvariable dient dem Modell als **Ground Truth (Label)**

---

### Schritt 6: Modelltraining

Ein Machine-Learning-Modell (z. B. **Logistische Regression**) wird trainiert:

- Eingabe: `Risiko_Score`
- Ziel: `ist_krank`
- Das Modell lernt, ab welchem Score das Risiko signifikant ansteigt

---

### Schritt 7: Modellauswertung

Nach dem Training wird das Modell bewertet:

- **Classification Report** mit Metriken wie Genauigkeit, Präzision, Recall
- **Confusion Matrix** zeigt richtig/falsch klassifizierte Fälle
- Das hilft, die **Qualität der Vorhersage** realistisch einzuschätzen

---

### Fazit:

Für die eigentliche Machine-Learning-Analyse benötigst du:

- ✅ den berechneten `Risiko_Score` (ab Schritt 4)
- ✅ die Zielvariable `ist_krank` (ab Schritt 5)
- ✅ das Training und die Bewertung des Modells (Schritte 6–7)

Die vorherigen Schritte (1–3) sind wichtig für die Datenvorbereitung – aber der ML-Prozess startet inhaltlich erst mit Schritt 4.



## Was sagt das Machine-Learning-Modell mit dem Risiko-Score aus?

Das trainierte Modell (z. B. **logistische Regression**) versucht zu **lernen**, ob eine Person ein erhöhtes Risiko für chronische Erkrankungen wie **Übergewicht, Bluthochdruck oder Diabetes** hat – und zwar **basierend auf dem Risiko-Score**.

---

### Was macht das Modell konkret?

Das Modell untersucht, **wie stark der Risiko-Score mit der Wahrscheinlichkeit für Krankheit zusammenhängt**.

> Beispiel:  
> Wenn eine Person einen Risiko-Score von 3 hat – also 3 Grenzwerte überschreitet –  
> wie wahrscheinlich ist es, dass sie zu einer Risikogruppe gehört?

---

### Das bedeutet:

| Fall                                     | Bedeutung                                        |
|------------------------------------------|--------------------------------------------------|
| **Hoher Score → hohe Wahrscheinlichkeit** | Person hat wahrscheinlich eine chronische Krankheit |
| **Niedriger Score → niedrige Wahrscheinlichkeit** | Person ist wahrscheinlich gesund |

Das Modell lernt also eine **mathematische Beziehung** zwischen Score und Krankheitsstatus.

---

### Wie gut ist das Modell?

Das Modell liefert mehrere Kennzahlen:

| Kennzahl             | Bedeutung                                                   |
|----------------------|-------------------------------------------------------------|
| **Accuracy**         | Wie viele Vorhersagen insgesamt korrekt waren               |
| **Precision**        | Anteil der korrekt vorhergesagten Kranken unter allen als krank eingestuften Personen |
| **Recall**           | Anteil der erkannten Kranken unter allen tatsächlich Kranken |
| **Confusion Matrix** | Zeigt korrekt und falsch klassifizierte Fälle grafisch an    |

---

### Warum ist das nützlich?

- Der Risiko-Score reicht als **einfaches Eingabemerkmal**  
- Das Modell kann auch **neue Personen automatisiert bewerten**  
- Ideal für **digitale Beratung, Prävention oder Priorisierung**  
- Es macht Zusammenhänge **quantifizierbar und erklärbar**

---

### Fazit:

> Das Modell sagt aus, **ab welchem Risiko-Score das Krankheitsrisiko signifikant steigt** – und wie zuverlässig diese Einschätzung ist.  
> Es ist ein **erster Schritt zu datengestützter Prävention**.

---


**ROC- / AUC-Kurve**

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# 1. Wahrscheinlichkeiten anstelle von Klassen vorhersagen
y_prob = model.predict_proba(X_test)[:, 1]  # Wahrscheinlichkeit für Klasse 1 (krank)

# 2. ROC-Kurve berechnen
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# 3. AUC berechnen
auc = roc_auc_score(y_test, y_prob)

# 4. Plot der ROC-Kurve
plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, label=f"ROC-Kurve (AUC = {auc:.3f})", color='darkorange')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Zufall (AUC = 0.5)')
plt.xlabel("False Positive Rate (1 - Spezifität)")
plt.ylabel("True Positive Rate (Recall)")
plt.title("ROC-Kurve: Risiko-Score → Krankheitsstatus")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()


**Interpretation:**

- Die ROC-Kurve zeigt alle möglichen Schwellenwerte – nicht nur bei 0.5.

- Die AUC-Zahl sagt direkt, wie gut das Modell insgesamt unterscheidet.

- Ideal zur Bewertung eines einfachen Risiko-Scores als Vorhersageinstrument.

**Verknüpfung des Risiko-Scores und des Machine-Learning-Modells, um konkrete personalisierte Ernährungsempfehlungen abzuleiten**

--> Aus dem vorhergesagten Krankheitsrisiko + den individuellen Ernährungsmustern konkrete Lebensmittel-Empfehlungen ableiten.

## Wie leite ich aus dem ML-Modell konkrete Ernährungsempfehlungen ab?

Nachdem du mit einem Machine-Learning-Modell das Krankheitsrisiko vorhergesagt hast, kannst du dieses Risiko zusammen mit den erkannten Ernährungsmustern nutzen, um **personalisierte Ernährungsempfehlungen** zu geben.

---

### 1. Vorhersage machen

Das Modell berechnet für jede Person:

> **„Wie hoch ist die Wahrscheinlichkeit, dass diese Person krank ist?“**  
> ➤ z. B. mit `model.predict_proba(X)` oder `model.predict(X)`

---

### 2. Nährstoffprofil prüfen

Unabhängig vom Modell weißt du aus den Daten:

| Nährstoffüberschreitung  | Bedeutung                                 |
|---------------------------|--------------------------------------------|
| `zu_viel_Kalorien`        | Person nimmt zu viel Energie auf           |
| `zu_viel_Fett`            | Hoher Fettkonsum                          |
| `zu_viel_Zucker`          | Zu hoher Zuckerkonsum                     |
| `zu_viel_Natrium`         | Salzaufnahme über WHO-Grenzwert           |

Diese Infos sind die Basis für individuelle Empfehlungen.

---

### 3. Regelbasierte Empfehlungen verknüpfen

Je nach Muster und Risikogruppe (z. B. hoher Risiko-Score oder hohe Modellwahrscheinlichkeit) kannst du gezielt Empfehlungen ableiten:

| Ernährungsmuster             | Empfehlung (Beispiel)                                                                 |
|-----------------------------|----------------------------------------------------------------------------------------|
| zu viel Kalorien + Fett     | 🥦 Mehr Gemüse, 🐟 Fisch, 🥗 Vollkorn, ❌ Fast Food & Wurst reduzieren                   |
| zu viel Zucker              | 🍓 Frisches Obst, 💧 Wasser statt Softdrinks, ❌ Süßigkeiten meiden                     |
| zu viel Natrium             | 🥔 Frisch kochen, ❌ Fertigprodukte meiden, 🥑 Kaliumreiche Kost bevorzugen            |
| hoher Risiko-Score (z. B. ≥ 3) | 🔴 Dringende Umstellung empfohlen – Ernährungsberatung oder Interventionsprogramm    |

---

Mit dieser Kombination aus **ML-gestützter Risikoeinschätzung** und **regelbasierter Empfehlung** entsteht ein **einfaches, aber leistungsfähiges Entscheidungssystem**, das personalisierte Empfehlungen auf Basis realer Daten liefert.


In [ ]:
# 1. CSV-Dateien lokal einlesen
diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv")
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

# Relevante Spalten
diet_cols = ['SEQN', 'DR1TKCAL', 'DR1TSUGR', 'DR1TTFAT', 'DR1TSODI']
examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']
questionnaire_cols = ['SEQN', 'DIQ010']

# Auswahl
diet_small = diet[diet_cols]
examination_small = examination[examination_cols]
questionnaire_small = questionnaire[questionnaire_cols]

# Zusammenführen
df = diet_small.merge(examination_small, on="SEQN", how="inner")
df = df.merge(questionnaire_small, on="SEQN", how="inner")

# NaNs entfernen und Kopie machen
df_clean = df.dropna().copy()

# Gesundheitsstatus
def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Keine der drei Diagnosen'

df_clean['Gesundheitsstatus'] = df_clean.apply(classify_health, axis=1)

# Grenzwertüberschreitungen
df_clean['zu_viel_Kalorien'] = df_clean['DR1TKCAL'] > 2500
df_clean['zu_viel_Fett'] = df_clean['DR1TTFAT'] > 75
df_clean['zu_viel_Zucker'] = df_clean['DR1TSUGR'] > 50
df_clean['zu_viel_Natrium'] = df_clean['DR1TSODI'] > 2000

# Risiko-Score
df_clean['Risiko_Score'] = (
    df_clean[['zu_viel_Kalorien', 'zu_viel_Fett', 'zu_viel_Zucker', 'zu_viel_Natrium']].sum(axis=1) +
    ((df_clean['zu_viel_Kalorien']) & (df_clean['zu_viel_Fett'])).astype(int)
)

# Empfehlungen generieren
def generate_recommendation(row):
    tips = []
    if row['zu_viel_Kalorien'] and row['zu_viel_Fett']:
        tips.append("➡ Weniger fett- & kalorienreiche Speisen (z. B. Fast Food, Wurst).")
        tips.append("✅ Mehr Gemüse, Fisch, Vollkornprodukte.")
    if row['zu_viel_Zucker']:
        tips.append("➡ Zucker reduzieren: Softdrinks & Süßigkeiten meiden.")
        tips.append("✅ Wasser, ungesüßter Tee, frisches Obst bevorzugen.")
    if row['zu_viel_Natrium']:
        tips.append("➡ Salz reduzieren: Fertigprodukte vermeiden.")
        tips.append("✅ Frisch kochen, kaliumreiche Lebensmittel wählen.")
    if row['Risiko_Score'] >= 4:
        tips.append("⚠️ Hohes Risiko – Bitte Ernährung grundlegend überdenken.")
    elif row['Risiko_Score'] >= 2:
        tips.append("🟡 Mittleres Risiko – Verbesserungen empfohlen.")
    else:
        tips.append("🟢 Geringes Risiko – aktuelles Verhalten beibehalten.")
    return " ".join(tips)

df_clean['Empfehlung'] = df_clean.apply(generate_recommendation, axis=1)

# Ergebnis anzeigen
df_clean[['SEQN', 'Gesundheitsstatus', 'Risiko_Score', 'Empfehlung']].head(10)

## Analyseerklärung & Interpretation

### Was wurde gemacht?

Auf Basis der NHANES-Daten wurden individuelle Ernährungsmuster analysiert und daraus **personalisierte Ernährungsempfehlungen** abgeleitet. Dabei wurden folgende Schritte durchgeführt:

1. **Datenbereinigung & Zusammenführung:**
   - Relevante Ernährungsdaten (z. B. Kalorien-, Zucker-, Fett- und Natriumaufnahme) wurden mit Gesundheitsdaten (Blutdruck, BMI, Diabetesdiagnose) kombiniert.

2. **Klassifikation des Gesundheitsstatus:**
   - Jede Person wurde einer Kategorie zugeordnet:
     - `Diabetes`, `Bluthochdruck`, `Übergewicht` oder `Keine der drei Diagnosen`.

3. **Nährstoffbewertung:**
   - Für jede Person wurde geprüft, ob die individuelle Nährstoffzufuhr über den folgenden Grenzwerten liegt:
     - **Kalorien** > 2500 kcal  
     - **Fett** > 75 g  
     - **Zucker** > 50 g  
     - **Natrium** > 2000 mg

4. **Risiko-Score-Berechnung:**
   - Je mehr Grenzwerte überschritten wurden, desto höher der Score.
   - Eine zusätzliche Gewichtung wurde gegeben, wenn sowohl Kalorien als auch Fett überschritten wurden (kombiniertes Überernährungsmuster).

5. **Personalisierte Empfehlungen:**
   - Basierend auf Ernährungsmuster und Risiko-Score wurden automatisiert Empfehlungen generiert (z. B. Reduktion von Zucker, Fett, Salz).

---

### Interpretation der Ergebnisse

- Personen mit einem **hohen Risiko-Score (≥ 4)** erhalten eine **dringende Empfehlung zur Ernährungskorrektur**, da sie mehrere Grenzwerte überschreiten.
- Personen mit **mittlerem Risiko (2–3)** erhalten gezielte Hinweise zur Verbesserung ihrer Ernährungsgewohnheiten.
- Personen mit **geringem Risiko (0–1)** erhalten positives Feedback und den Hinweis, ihr Verhalten beizubehalten.

Diese Analyse zeigt, wie man aus **realen Gesundheits- und Ernährungsdaten** evidenzbasierte, individualisierte Ernährungsempfehlungen ableiten kann. Dabei wird sowohl das **aktuelle Gesundheitsrisiko** als auch das **konkrete Ernährungsverhalten** berücksichtigt.

---

### Nächste Schritte (optional):

- Visualisierung der Empfehlungen nach Gesundheitsstatus
- Export der Empfehlungen als Excel/CSV für ärztliche Beratung
- Clustering von Ernährungsmustern (z. B. k-Means)
- Verknüpfung mit Machine-Learning-Modellen zur Risikoabschätzung


**Verknüpfung mit konkreten Lebensmitteln (Lebensmittel die empfohlen werden oder gemieden werden sollen)**

In [ ]:
def classify_pattern(row):
    if row['zu_viel_Kalorien'] and row['zu_viel_Fett']:
        return 'Fett- & kalorienreich'
    elif row['zu_viel_Zucker']:
        return 'Zuckerreich'
    elif row['zu_viel_Natrium']:
        return 'Salzreich'
    else:
        return 'Unauffällig'

df_clean['Ernährungsmuster'] = df_clean.apply(classify_pattern, axis=1)



lebensmittel_empfehlungen = {
    'Fett- & kalorienreich': {
        'Empfohlen': "Brokkoli, Kartoffeln, Vollkornreis, Hülsenfrüchte, Lachs",
        'Meiden': "Fast Food, Wurst, Käse, Frittierte Speisen"
    },
    'Zuckerreich': {
        'Empfohlen': "Beeren, Äpfel, Wasser, Ungesüßter Tee",
        'Meiden': "Softdrinks, Süßigkeiten, gezuckertes Müsli"
    },
    'Salzreich': {
        'Empfohlen': "Frisches Gemüse, Spinat, Banane, ungesalzene Nüsse",
        'Meiden': "Chips, Fertiggerichte, Wurst, Instantnudeln"
    },
    'Unauffällig': {
        'Empfohlen': "Ausgewogene Mischkost – Obst, Gemüse, Vollkorn",
        'Meiden': "Keine besonderen Einschränkungen"
    }
}

# Zuordnung anwenden
df_clean['Lebensmittel_Empfohlen'] = df_clean['Ernährungsmuster'].map(lambda x: lebensmittel_empfehlungen[x]['Empfohlen'])
df_clean['Lebensmittel_Meiden'] = df_clean['Ernährungsmuster'].map(lambda x: lebensmittel_empfehlungen[x]['Meiden'])

df_clean[['SEQN', 'Gesundheitsstatus', 'Ernährungsmuster', 'Lebensmittel_Empfohlen', 'Lebensmittel_Meiden']].head(10)


**Individualisierte Empfehlung für jede Person**

Muss als Code rein --> aber zuerst muss ich ins Read.me folgendes einfügen, weil sonst das Speichern auf GitHub nicht funktioniert (siehe untenfolgendes Markdown-Feld)

# interaktives Ernährungsempfehlungsformular
# import ipywidgets as widgets
# from IPython.display import display, Markdown

# Eingabefelder
gewicht = widgets.FloatText(description='Gewicht (kg):', value=70)
groesse = widgets.FloatText(description='Größe (cm):', value=170)
alter = widgets.IntSlider(description='Alter:', min=10, max=100, value=30)
kalorien = widgets.IntSlider(description='Kalorien (kcal):', min=1000, max=5000, step=100, value=2500)
fett = widgets.IntSlider(description='Fett (g):', min=0, max=200, value=70)
zucker = widgets.IntSlider(description='Zucker (g):', min=0, max=150, value=40)
natrium = widgets.IntSlider(description='Natrium (mg):', min=0, max=5000, step=100, value=1800)

button = widgets.Button(description='Empfehlung anzeigen', button_style='success')
output = widgets.Output()

# Funktion zur Auswertung
def berechne_und_empfehle(b):
    with output:
        output.clear_output()
        bmi = gewicht.value / (groesse.value / 100) ** 2

        zu_viel_kcal = kalorien.value > 2500
        zu_viel_fett = fett.value > 75
        zu_viel_zucker = zucker.value > 50
        zu_viel_natrium = natrium.value > 2000

        if zu_viel_kcal and zu_viel_fett:
            muster = 'Fett- & kalorienreich'
        elif zu_viel_zucker:
            muster = 'Zuckerreich'
        elif zu_viel_natrium:
            muster = 'Salzreich'
        else:
            muster = 'Unauffällig'

        lebensmittel_empfehlungen = {
            'Fett- & kalorienreich': {
                'Empfohlen': "Brokkoli, Kartoffeln, Vollkornreis, Hülsenfrüchte, Lachs",
                'Meiden': "Fast Food, Wurst, Käse, Frittierte Speisen"
            },
            'Zuckerreich': {
                'Empfohlen': "Beeren, Äpfel, Wasser, Ungesüßter Tee",
                'Meiden': "Softdrinks, Süßigkeiten, gezuckertes Müsli"
            },
            'Salzreich': {
                'Empfohlen': "Frisches Gemüse, Spinat, Banane, ungesalzene Nüsse",
                'Meiden': "Chips, Fertiggerichte, Wurst, Instantnudeln"
            },
            'Unauffällig': {
                'Empfohlen': "Ausgewogene Mischkost – Obst, Gemüse, Vollkorn",
                'Meiden': "Keine besonderen Einschränkungen"
            }
        }

        emp = lebensmittel_empfehlungen[muster]['Empfohlen']
        mei = lebensmittel_empfehlungen[muster]['Meiden']

        display(Markdown(f"""
### 🔎 Ergebnis für deine Eingaben
- **BMI:** {bmi:.1f}
- **Ernährungsmuster:** `{muster}`

**✅ Empfohlene Lebensmittel:**  
{emp}

**⛔ Zu meiden:**  
{mei}
"""))

# Button verknüpfen und Formular anzeigen
button.on_click(berechne_und_empfehle)
formular = widgets.VBox([gewicht, groesse, alter, kalorien, fett, zucker, natrium, button, output])
display(formular)


# 🧪 Interaktive Ernährungsempfehlung

Dieses Projekt ermöglicht es, auf Basis persönlicher Angaben (Größe, Gewicht, Nährstoffaufnahme) eine **automatische Klassifikation des Ernährungsmusters** vorzunehmen – mit konkreten Empfehlungen zur Ernährung.

## ▶️ Direkt ausprobieren in Google Colab

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/DEIN_GITHUB_NAME/DEIN_REPO_NAME/blob/main/analyse_interaktiv.ipynb)

## 📦 Anforderungen
- `ipywidgets`
- `IPython.display`

## 💡 Funktionsweise
- Berechnet den BMI
- Prüft Schwellenwerte für Kalorien, Zucker, Fett, Natrium
- Gibt passende Empfehlungen zu Lebensmitteln aus


In [ ]:
def berechne_und_empfehle_mit_ml(gewicht, groesse, alter, kalorien, fett, zucker, natrium, model):
    bmi = gewicht / (groesse / 100) ** 2


    print("++++++++++" + str(gesamtumsatz) + "++++++++")
    zu_viel_kcal = kalorien > gesamtumsatz#2500
    zu_viel_fett = fett > 75
    zu_viel_zucker = zucker > 50
    zu_viel_natrium = natrium > 2000

    # Risiko-Score berechnen
    risiko_score = int(zu_viel_kcal) + int(zu_viel_fett) + int(zu_viel_zucker) + int(zu_viel_natrium)
    if zu_viel_kcal and zu_viel_fett:
        risiko_score += 1

    # ML-Modell Vorhersage
#    prediction = model.predict([[risiko_score]])[0]
#    proba = model.predict_proba([[risiko_score]])[0][1]


    input_df = pd.DataFrame([[risiko_score]], columns=["Risiko_Score"])

    prediction = model.predict(input_df)[0]
    proba = model.predict_proba(input_df)[0][1]


    # Ernährungsmuster bestimmen
    if zu_viel_kcal and zu_viel_fett:
        muster = 'Fett- & kalorienreich'
    elif zu_viel_zucker:
        muster = 'Zuckerreich'
    elif zu_viel_natrium:
        muster = 'Salzreich'
    else:
        muster = 'Unauffällig'

    # Empfehlungen
    lebensmittel_empfehlungen = {
        'Fett- & kalorienreich': {
            'Empfohlen': "Brokkoli, Kartoffeln, Vollkornreis, Hülsenfrüchte, Lachs",
            'Meiden': "Fast Food, Wurst, Käse, Frittierte Speisen"
        },
        'Zuckerreich': {
            'Empfohlen': "Beeren, Äpfel, Wasser, Ungesüsster Tee",
            'Meiden': "Softdrinks, Süssigkeiten, gezuckertes Müsli"
        },
        'Salzreich': {
            'Empfohlen': "Frisches Gemüse, Spinat, Banane, ungesalzene Nüsse",
            'Meiden': "Chips, Fertiggerichte, Wurst, Instantnudeln"
        },
        'Unauffällig': {
            'Empfohlen': "Ausgewogene Mischkost – Obst, Gemüse, Vollkorn",
            'Meiden': "Keine besonderen Einschränkungen"
        }
    }

    emp = lebensmittel_empfehlungen[muster]['Empfohlen']
    mei = lebensmittel_empfehlungen[muster]['Meiden']

    print(f"\n🔎 Ergebnis für deine Eingaben:")
    print(f"- BMI: {bmi:.1f}")
    print(f"- Ernährungsmuster: {muster}")
    print(f"- Risiko-Score: {risiko_score}")
    print(f"- ML-Vorhersage: {'Krankheitsrisiko erkannt' if prediction == 1 else 'Kein auffälliges Risiko'}")
    print(f"- Wahrscheinlichkeit für Risiko: {proba:.2%}")
    print(f"\n✅ Empfohlene Lebensmittel:\n{emp}")
    print(f"\n⛔ Zu meiden:\n{mei}")


In [ ]:
werte = get_user_inputs()
berechne_und_empfehle_mit_ml(*werte, model)


#Murats Teil

In [ ]:
# === Päckli 1a: Meals-Macros Cleaned ===
print("\n📦 Lade df_meals ...")
df_meals = pd.read_csv(
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/archive/detailed_meals_macros_CLEANED.csv"
)
df_meals.info()
display(df_meals.head(3))

# === Päckli 1b: FOOD Nutrition ===
print("\n📦 Lade df_nutrition ...")
df_nutrition = pd.read_csv(
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/archive/Food_and_Nutrition__.csv"
)
df_nutrition.info()
display(df_nutrition.head(3))

# === Päckli 2: FOOD DATA GROUPS ===
print("\n📦 Lade food_groups_combined ...")
food_data_urls = [
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP1.csv",
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP2.csv",
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP3.csv",
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP4.csv",
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/Kaggle/FINAL%20FOOD%20DATASET/FOOD-DATA-GROUP5.csv",
]
food_data_dfs = [pd.read_csv(url) for url in food_data_urls]
food_groups_combined = pd.concat(food_data_dfs, ignore_index=True)

food_groups_combined.info()
display(food_groups_combined.head(3))

# === Päckli 3: BLV-Referenzwerte ===
print("\n📦 Lade blv_nutrient_requirements ...")
blv_nutrient_requirements = pd.read_csv(
    "https://raw.githubusercontent.com/M2xAy/ffhs-datensaetze/refs/heads/main/Datasets/BLV/Nutrient_requirements_CH-2022.csv"
)
blv_nutrient_requirements.info()
display(blv_nutrient_requirements.head(3))


print("\n📊 Übersicht aller geladenen DataFrames:")

dataframes = {
    "df_meals": df_meals,
    "df_nutrition": df_nutrition,
    "food_groups_combined": food_groups_combined,
    "blv_nutrient_requirements": blv_nutrient_requirements
}

for name, df in dataframes.items():
    print(f"- {name}: {df.shape[0]} Zeilen × {df.shape[1]} Spalten")



In [ ]:
# === 1. Fehlende Werte prüfen ===
missing_counts = df_meals.isnull().sum()
missing_total = df_meals.isnull().any(axis=1).sum()

print(f"\n# 🔍 Anzahl ungültiger Zeilen mit fehlenden Werten: {missing_total}")
print("\n# 🔍 Fehlende Werte pro Spalte:")
print(missing_counts[missing_counts > 0])

# === 2. Entferne Zeilen mit fehlenden Werten in relevanten Spalten ===
critical_columns = ["Breakfast Carbohydrates"]
df_meals.dropna(subset=critical_columns, inplace=True)

# Optional: Falls du *alle* NaN-Zeilen entfernen willst:
# df_meals.dropna(inplace=True)

# === 3. Duplikate entfernen ===
df_meals.drop_duplicates(subset=[col for col in df_meals.columns if col != "Disease_List"], inplace=True)

# === 4. Krankheiten vorbereiten ===
df_meals["Disease_List"] = df_meals["Disease"].fillna("").apply(lambda x: [d.strip() for d in x.split(",") if d.strip()])
df_meals["Disease_Count"] = df_meals["Disease_List"].apply(len)
df_meals["Disease_Tuple"] = df_meals["Disease_List"].apply(tuple)

# === 5. Zusammenfassung ===
print(f"\n# ✅ Anzahl verbleibender (gültiger) Datensätze: {len(df_meals)}")
print(f"# 🧹 Bereinigt: {missing_total - df_meals.isnull().any(axis=1).sum()} ungültige Zeilen entfernt")


In [ ]:
# === 1. Fehlende Werte prüfen ===
missing_counts = df_nutrition.isnull().sum()
missing_total = df_nutrition.isnull().any(axis=1).sum()

print(f"\n# 🔍 [df_nutrition] Anzahl ungültiger Zeilen mit fehlenden Werten: {missing_total}")
print(missing_counts[missing_counts > 0])

# === 2. Entferne alle Zeilen mit fehlenden Werten
df_nutrition.dropna(inplace=True)

# === 3. Duplikate entfernen
df_nutrition.drop_duplicates(inplace=True)

# === 4. Zusammenfassung
print(f"# ✅ Verbleibende Datensätze: {len(df_nutrition)}")
print(f"# 🧹 Bereinigt: {missing_total - df_nutrition.isnull().any(axis=1).sum()} ungültige Zeilen entfernt")


In [ ]:
# === 1. Fehlende Werte prüfen ===
missing_counts = food_groups_combined.isnull().sum()
missing_total = food_groups_combined.isnull().any(axis=1).sum()

print(f"\n# 🔍 [food_groups_combined] Anzahl ungültiger Zeilen mit fehlenden Werten: {missing_total}")
print(missing_counts[missing_counts > 0])

# === 2. Entferne alle Zeilen mit fehlenden Werten
food_groups_combined.dropna(inplace=True)

# === 3. Duplikate entfernen
food_groups_combined.drop_duplicates(inplace=True)

# === 4. Zusammenfassung
print(f"# ✅ Verbleibende Datensätze: {len(food_groups_combined)}")
print(f"# 🧹 Bereinigt: {missing_total - food_groups_combined.isnull().any(axis=1).sum()} ungültige Zeilen entfernt")


In [ ]:
# === 1. Fehlende Werte prüfen ===
# 2. Nur Zeilen entfernen, die wirklich KOMPLETT leer sind
blv_nutrient_requirements.dropna(how="all", inplace=True)

# 3. Spaltennamen säubern
blv_nutrient_requirements.columns = (
    blv_nutrient_requirements.columns
    .str.replace(r'\r\n', ' ', regex=True)
    .str.replace(r'[^\w\s\(\)/%.-]', '', regex=True)
    .str.strip()
)

# 4. Jetzt NUR gezielt Sonderzeichen ersetzen (nicht sofort alles)
# Noch KEIN dropna() mehr!
print(f"✅ Zeilen nach Reparatur: {len(blv_nutrient_requirements)}")
display(blv_nutrient_requirements.head(10))

In [ ]:
# === 1.1 Feature Enginerring ===
# === 🧬 Krankheiten vorbereiten ===
df_meals["Disease_List"] = df_meals["Disease"].fillna("").apply(lambda x: [d.strip() for d in x.split(",") if d.strip()])
df_meals["Disease_Count"] = df_meals["Disease_List"].apply(len)
df_meals["Has_Chronic"] = df_meals["Disease_List"].apply(
    lambda lst: any(k in lst for k in ["Diabetes", "Hypertension", "Heart Disease", "Kidney Disease"])
)
df_meals["Multiple_Conditions"] = (df_meals["Disease_Count"] > 1).astype(int)
df_meals["Target_Chronic"] = df_meals["Has_Chronic"].astype(int)

# === 📏 Proteinbedarf nach BLV (0.8 g/kg Körpergewicht) ===
blv_protein = 0.8
df_meals["BLV_Protein_Need"] = df_meals["Weight"] * blv_protein

# === 🔹 Makroverhältnisse ===
df_meals["Protein_per_100kcal"] = df_meals["Protein"] / df_meals["Calories"] * 100
df_meals["Fat_to_Protein_Ratio"] = df_meals["Fat"] / (df_meals["Protein"] + 1e-6)
df_meals["Carb_to_Fat_Ratio"] = df_meals["Carbohydrates"] / (df_meals["Fat"] + 1e-6)
df_meals["Sugar_per_Carb"] = df_meals["Sugar"] / (df_meals["Carbohydrates"] + 1e-6)

# === 🔸 Zielüberschreitungen ===
df_meals["Calories_exceed_target"] = (df_meals["Calories"] > df_meals["Daily Calorie Target"]).astype(int)
df_meals["Sugar_excess"] = (df_meals["Sugar"] > 50).astype(int)
df_meals["Protein_deficit_ratio"] = df_meals["Protein"] / (df_meals["BLV_Protein_Need"] + 1e-6)

# === ⏰ Tagesstruktur-Features ===
df_meals["Protein_Lunch_vs_Dinner"] = df_meals["Lunch Protein"] / (df_meals["Dinner Protein.1"] + 1e-6)
df_meals["Snack_Energy_Share"] = df_meals["Snacks Calories"] / df_meals["Daily Calorie Target"]
df_meals["Carb_ratio_breakfast"] = df_meals["Breakfast Carbohydrates"] / (df_meals["Carbohydrates"] + 1e-6)

# === 📊 Meta-Features ===
df_meals["Protein_std"] = df_meals[["Breakfast Protein", "Lunch Protein", "Dinner Protein.1", "Snacks Protein"]].std(axis=1)
df_meals["Macronutrient_Diversity"] = df_meals[["Protein", "Fat", "Carbohydrates", "Fiber", "Sugar"]].gt(0).sum(axis=1)

# === ⚠️ Risikoindikator ===
df_meals["Risk_Flag"] = (
    (df_meals["Sugar_excess"] == 1) |
    (df_meals["Calories_exceed_target"] == 1) |
    (df_meals["Protein_deficit_ratio"] < 0.8)
).astype(int)

# ✅ Ergebnisvorschau (optional anzeigen)
features_to_show = [
    "Protein_per_100kcal", "Fat_to_Protein_Ratio", "Carb_to_Fat_Ratio", "Sugar_per_Carb",
    "Calories_exceed_target", "Sugar_excess", "Protein_deficit_ratio",
    "Protein_Lunch_vs_Dinner", "Snack_Energy_Share", "Carb_ratio_breakfast",
    "Protein_std", "Macronutrient_Diversity",
    "Disease_Count", "Has_Chronic", "Multiple_Conditions", "Target_Chronic", "Risk_Flag"
]



#########Feature Eng. für KAQ1#########
# Neues Feature: Protein pro 100 kcal
df_meals["Protein_per_100kcal"] = df_meals["Protein"] / df_meals["Calories"] * 100

# Neues Feature: Fett zu Protein Verhältnis
df_meals["Fat_to_Protein_Ratio"] = df_meals["Fat"] / (df_meals["Protein"] + 1e-6)  # Schutz gegen Division durch 0

# Neues Feature: Zuckeranteil an Kohlenhydraten
df_meals["Sugar_per_Carb"] = df_meals["Sugar"] / (df_meals["Carbohydrates"] + 1e-6)

# Neues Feature: Snack-Energie-Anteil
df_meals["Snack_Energy_Share"] = df_meals["Snacks Calories"] / (df_meals["Daily Calorie Target"] + 1e-6)

# Neues Feature: Kohlenhydratanteil Frühstück
df_meals["Carb_ratio_breakfast"] = df_meals["Breakfast Carbohydrates"] / (df_meals["Carbohydrates"] + 1e-6)


In [ ]:
# =====================================
# Schritt 3.1: Explorative Datenanalyse (EDA)
# =====================================

# ------------------------------
# 3.1.1 Verteilung der Makronährstoffe
# ------------------------------
macro_columns = ["Calories", "Protein", "Fat", "Sugar", "Sodium", "Carbohydrates"]
fig1, axes = plt.subplots(len(macro_columns), 1, figsize=(10, 20))

for i, col in enumerate(macro_columns):
    sns.histplot(df_meals[col], kde=True, ax=axes[i], color='skyblue')
    axes[i].set_title(f"Verteilung von {col}", fontsize=12)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Häufigkeit")

plt.tight_layout()
plt.show()

# ------------------------------
# 3.1.2 Häufigkeit von Krankheiten
# ------------------------------
plt.figure(figsize=(10, 6))
df_meals["Disease"].value_counts().plot(kind="bar", color="salmon")
plt.title("Häufigkeit von Krankheiten (kombiniert)", fontsize=13)
plt.xlabel("Krankheit(en)")
plt.ylabel("Anzahl Personen")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# ------------------------------
# 3.1.3 Vergleich Kalorienaufnahme nach Geschlecht
# ------------------------------
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_meals, x="Gender", y="Calories", palette="pastel")
plt.title("Kalorienaufnahme nach Geschlecht", fontsize=13)
plt.xlabel("Geschlecht")
plt.ylabel("Kalorien")
plt.tight_layout()
plt.show()




# ------------------------------
# 3.1.4 Deskriptive Statistik mit erweiterten Kennzahlen
# ------------------------------
columns = ["Calories", "Protein", "Fat", "Sugar", "Sodium", "Carbohydrates"]
desc_stats = df_meals[columns].describe().T  # count, mean, std, min, 25%, 50%, 75%, max
desc_stats["median"] = df_meals[columns].median()
desc_stats["skewness"] = df_meals[columns].skew()
desc_stats["kurtosis"] = df_meals[columns].kurt()

# Ausgabe als Tabelle
print("Deskriptive Statistik (inkl. Median, Schiefe & Kurtosis):\n")
print(desc_stats.round(2))

# ------------------------------
# 3.1.5 Korrelationen (Heatmap)
# ------------------------------
correlation_matrix = df_meals[columns].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korrelation zwischen Makronährstoffen", fontsize=13)
plt.tight_layout()
plt.show()


# ------------------------------
# 3.1.6 Deskriptive Statistik mit erweiterten Kennzahlen
# ------------------------------
columns = ["Calories", "Protein", "Fat", "Sugar", "Sodium", "Carbohydrates"]
desc_stats = df_meals[columns].describe().T  # count, mean, std, min, 25%, 50%, 75%, max
desc_stats["median"] = df_meals[columns].median()
desc_stats["skewness"] = df_meals[columns].skew()
desc_stats["kurtosis"] = df_meals[columns].kurt()

# Ausgabe als Tabelle
print("Deskriptive Statistik (inkl. Median, Schiefe & Kurtosis):\n")
print(desc_stats.round(2))



In [ ]:
# =====================================
# Schritt 3.2: Ernährungsmuster identifizieren
# =====================================

# =====================================
# Analyse KAQ 1: Welche Ernährungsmuster lassen sich aus individuellen Makronährstoffprofilen ableiten – und welche Krankheitsrisiken sind damit verbunden?
# Methode: KMeans-Clusteranalyse auf Protein, Fett, Zucker, Kalorien, etc.
# Ziel: Ernährungstypen identifizieren
# Zusatzanalyse: Krankheitsverteilung je Cluster
# Visualisierung: Clusterplot, Balkendiagramm Krankheitsprävalenz
# =====================================


# 3. Clustering (KAQ 1) - 1
# Clustering (KAQ 1) - 1 mit Beschriftung
features_kaq1 = [
    "Protein_per_100kcal", "Fat_to_Protein_Ratio", "Sugar_per_Carb",
    "Snack_Energy_Share", "Carb_ratio_breakfast", "Calories_exceed_target",
    "Protein_deficit_ratio"
]

df_kaq1 = df_meals[features_kaq1 + ["Target_Chronic", "Risk_Flag"]].dropna().copy()
# Neue Flags für Unausgewogenheit erstellen
df_kaq1["Under_Protein"] = (df_kaq1["Protein_deficit_ratio"] < 1.0).astype(int)
df_kaq1["High_Sugar_Carb"] = (df_kaq1["Sugar_per_Carb"] > 0.3).astype(int)
df_kaq1["High_Snacks"] = (df_kaq1["Snack_Energy_Share"] > 0.15).astype(int)
df_kaq1["Fat_Heavy"] = (df_kaq1["Fat_to_Protein_Ratio"] > 1.0).astype(int)
df_kaq1["Unausgewogen"] = (
    df_kaq1["Under_Protein"] +
    df_kaq1["High_Sugar_Carb"] +
    df_kaq1["High_Snacks"] +
    df_kaq1["Calories_exceed_target"] +
    df_kaq1["Fat_Heavy"]
)
df_kaq1["Unausgewogen_Flag"] = (df_kaq1["Unausgewogen"] >= 2).astype(int)





# Clustering + PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_kaq1[features_kaq1])
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_pca)

df_kaq1["Cluster"] = clusters
df_kaq1["PCA1"] = X_pca[:, 0]
df_kaq1["PCA2"] = X_pca[:, 1]

farben = {0: "green", 1: "orange", 2: "blue"}
farben_liste = df_kaq1["Cluster"].map(farben)
cluster_labels = {
    0: "🟢 Cluster 0:\nProteinreich\nNiedriges Risiko",
    1: "🟠 Cluster 1:\nSnacklastig\nMittleres Risiko",
    2: "🔵 Cluster 2:\nCarb-Frühstück\nHöchstes Risiko"
}

plt.figure(figsize=(10, 6))
plt.scatter(df_kaq1["PCA1"], df_kaq1["PCA2"], c=farben_liste, s=40, alpha=0.7)
for cluster_id, label in cluster_labels.items():
    x_mean = df_kaq1[df_kaq1["Cluster"] == cluster_id]["PCA1"].mean()
    y_mean = df_kaq1[df_kaq1["Cluster"] == cluster_id]["PCA2"].mean()
    plt.text(x_mean, y_mean, label, fontsize=10, weight='bold', ha='center', va='center',
             bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))
plt.title("KAQ 1: Ernährungsmuster-Clustering (beschriftet)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.grid(True)
plt.tight_layout()
plt.show()



# Definition der Basis-Nährstoff-Features
features_basic = ["Protein", "Fat", "Carbohydrates", "Sugar"]

# DataFrame für Basis-Clustering:
df_basic = df_meals[features_basic].dropna().copy()


# Clustering auf Basis-Nährstoffe mit Beschriftung
scaler = StandardScaler()
X_scaled_basic = scaler.fit_transform(df_basic)
pca_basic = PCA(n_components=2)
X_pca_basic = pca_basic.fit_transform(X_scaled_basic)
kmeans_basic = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters_basic = kmeans_basic.fit_predict(X_pca_basic)

df_basic["Cluster"] = clusters_basic
df_basic["PCA1"] = X_pca_basic[:, 0]
df_basic["PCA2"] = X_pca_basic[:, 1]

farben_basic = {0: "green", 1: "orange", 2: "blue"}
farben_liste_basic = df_basic["Cluster"].map(farben_basic)
cluster_labels_basic = {
    0: "🟢 Cluster 0:\nProteinreich",
    1: "🟠 Cluster 1:\nFett-/Zuckerreich",
    2: "🔵 Cluster 2:\nCarblastig"
}

plt.figure(figsize=(10, 6))
plt.scatter(df_basic["PCA1"], df_basic["PCA2"], c=farben_liste_basic, s=40, alpha=0.7)
for cluster_id, label in cluster_labels_basic.items():
    x_mean = df_basic[df_basic["Cluster"] == cluster_id]["PCA1"].mean()
    y_mean = df_basic[df_basic["Cluster"] == cluster_id]["PCA2"].mean()
    plt.text(x_mean, y_mean, label, fontsize=10, weight='bold', ha='center', va='center',
             bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))
plt.title("Clustering auf Protein, Fett, Carbs, Zucker (beschriftet)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.grid(True)
plt.tight_layout()
plt.show()




# Plotly HBar für KAQ 1 Features
cluster_summary = df_kaq1.groupby("Cluster")[features_kaq1].mean().reset_index()
df_melt = cluster_summary.melt(id_vars="Cluster", var_name="Feature", value_name="Wert")
fig = px.bar(df_melt, x="Wert", y="Feature", color="Cluster", orientation="h", barmode="group",
             color_discrete_map={0: "green", 1: "orange", 2: "blue"}, height=600,
             title="Ernährungsmuster je Cluster (features_kaq1)")
fig.update_layout(xaxis_title="Durchschnittlicher Wert", yaxis_title="Feature", legend_title="Cluster", template="plotly_white")
fig.show()

# Plotly HBar für Basis-Nährstoffe
cluster_summary_basic = df_basic.groupby("Cluster")[features_basic].mean().reset_index()
df_melt_basic = cluster_summary_basic.melt(id_vars="Cluster", var_name="Feature", value_name="Wert")
fig_basic = px.bar(df_melt_basic, x="Wert", y="Feature", color="Cluster", orientation="h", barmode="group",
                   color_discrete_map={0: "green", 1: "orange", 2: "blue"}, height=500,
                   title="Basis-Nährstoffe je Cluster")
fig_basic.update_layout(xaxis_title="Durchschnittlicher Wert (g)", yaxis_title="Basis-Nährstoff", legend_title="Cluster", template="plotly_white")
fig_basic.show()




# Anteil Unausgewogener je Cluster
unausgewogen_summary = df_kaq1.groupby("Cluster")[["Unausgewogen_Flag"]].mean().reset_index()
fig_unaus = px.bar(unausgewogen_summary, x="Cluster", y="Unausgewogen_Flag", color="Cluster",
                   color_discrete_map={0: "green", 1: "orange", 2: "blue"},
                   title="Anteil Unausgewogener Ernährung je Cluster",
                   labels={"Unausgewogen_Flag": "Anteil (%)"}, text_auto=".1%", height=400)
fig_unaus.update_layout(template="plotly_white", yaxis_tickformat=".0%")
fig_unaus.show()



In [ ]:
# =====================================
# Schritt 3.3: Zusammenhang zwischen Ernährung und Krankheitsrisiken
# =====================================

In [ ]:
# =====================================
# Schritt 3.4: Identifikation unausgewogener Ernährungsmuster
# =====================================

In [ ]:
# =====================================
# Schritt 3.5: Vergleich von Personen mit und ohne Krankheit
# =====================================

In [ ]:
# =====================================
# Schritt 3.6: Empfehlungen basierend auf Clusterbildung
# =====================================

In [ ]:
# =====================================
# Schritt 3: Explorative Analyse nach IBCS
# =====================================

# 🧠 Frage: Unterscheiden sich kranke und gesunde Personen im Zuckerkonsum?
# 🎯 Ziel: Visuelle Verteilung und Mittelwertunterschied zeigen

sns.boxplot(data=df_meals, x=df_meals["Disease_Count"] > 0, y="Sugar")
plt.title("Zuckerzufuhr bei Personen mit und ohne Krankheit")
plt.xlabel("Krankheit vorhanden?")
plt.ylabel("Zucker (g pro Tag)")
plt.grid(True)
plt.show()

# SAY-Prinzip (Kommentar zur Grafik)
print("Personen mit Krankheit weisen im Median einen leicht höheren Zuckerkonsum auf.")


In [ ]:
#Ernährungsmuster visuell erkennen (z. B. Cluster 1 = „Proteinreich und risikoarm“)
#Krankheitsrisiken vorhersagen (RandomForest auf Basis deiner Features)
#Datenqualität prüfen (z. B. ob Cluster mit Risk_Flag oder Target_Chronic übereinstimmen)
#Zielgruppen ableiten für personalisierte Ernährungsempfehlungen


# === 1. Feature-Auswahl ===
features = [
    "Protein_per_100kcal", "Fat_to_Protein_Ratio", "Carb_to_Fat_Ratio", "Sugar_per_Carb",
    "Calories_exceed_target", "Sugar_excess", "Protein_deficit_ratio",
    "Protein_Lunch_vs_Dinner", "Snack_Energy_Share", "Carb_ratio_breakfast",
    "Protein_std", "Macronutrient_Diversity"
]

# ✅ Nur vollständige Zeilen verwenden
df_cluster = df_meals[features + ["Target_Chronic"]].dropna().copy()

# === 2. Standardisieren ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster[features])

# === 3. PCA → 2D-Reduktion
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# === 4. KMeans Clustering
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_pca)

# === 5. Cluster + PCA in df_meals schreiben
df_meals_clean = df_meals.loc[df_cluster.index].copy()
df_meals_clean["Cluster"] = clusters
df_meals_clean["PCA1"] = X_pca[:, 0]
df_meals_clean["PCA2"] = X_pca[:, 1]

# === 6. Cluster-Zusammenfassung
cluster_summary = df_meals_clean.groupby("Cluster")[features].mean()

# === 7. Cluster-Visualisierung (PCA 2D)
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='Set2', s=40, alpha=0.7)
plt.title("PCA + KMeans Clustering der Ernährungsmuster")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.grid(True)
plt.legend(*scatter.legend_elements(), title="Cluster")
plt.tight_layout()
plt.show()

# === 8. Klassifikation: Target_Chronic
X = df_cluster[features]
y = df_cluster["Target_Chronic"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=22,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    n_jobs=1,  # statt -1, um Fehler zu vermeiden
    random_state=42
)

clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)

# === 9. Klassifikationsmetriken
report = classification_report(y_test, y_pred, output_dict=True)
metrics = pd.DataFrame(report).T

# === 10. Optional: Boxplot pro Cluster für ein Feature
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_meals_clean, x="Cluster", y="Protein_per_100kcal", palette="Set2")
plt.title("Protein pro 100kcal nach Cluster")
plt.grid(True)
plt.tight_layout()
plt.show()

# === 11. Ausgabe
print("\n📊 Cluster-Zusammenfassung:")
display(cluster_summary)

print("\n🎯 Klassifikationsmetriken:")
display(metrics)


In [ ]:
def vorhersage_chronisches_risiko(input_dict, model, feature_list):
    """
    Gibt eine Vorhersage für chronisches Risiko zurück (basierend auf Random Forest Modell).

    :param input_dict: dict – Eingabewerte für eine Person (z. B. {"Protein_per_100kcal": 5.0, ...})
    :param model: sklearn-Modell – z. B. trainierter RandomForestClassifier
    :param feature_list: list – Liste der Feature-Namen in richtiger Reihenfolge
    :return: Tuple (Label-Text, Wahrscheinlichkeit in %)
    """
    import pandas as pd

    # Eingabe in DataFrame umwandeln
    input_df = pd.DataFrame([input_dict])[feature_list]

    # Vorhersage & Wahrscheinlichkeit
    prediction = model.predict(input_df)[0]
    prob = model.predict_proba(input_df)[0][prediction]

    # Label interpretieren
    label_text = "chronisches Risiko vorhanden" if prediction == 1 else "kein chronisches Risiko"

    return label_text, round(prob * 100, 2)


In [ ]:
#Feature Enginerring nachträglich am 23.04.2025 von kay hinzugefügt
#Altersbereiche (age_min, age_max) zu extrahieren, Den passenden Protein-Referenzwert (oder andere Nährstoffe) je Person aus df_meals zu matchen, Eine Abweichung als Verhältnis (Protein / Protein_BLV) zu berechnen



# 2. Erste Zeile als Spaltenüberschriften setzen
blv_nutrient_requirements.columns = blv_nutrient_requirements.iloc[0]
blv_nutrient_requirements = blv_nutrient_requirements.drop(0)

# 3. Spalten korrekt benennen
blv_nutrient_requirements.rename(columns={
    blv_nutrient_requirements.columns[0]: "sex",
    blv_nutrient_requirements.columns[1]: "age_range"
}, inplace=True)

# 4. Altersbereich extrahieren
def parse_age_range(age_str):
    if pd.isna(age_str):
        return (None, None)
    age_str = str(age_str).strip().lower().replace("jahre", "").replace("years", "").replace("y", "")
    age_str = age_str.replace(">", "").replace("<", "").replace("+", "").replace("m", "").strip()
    try:
        if "to" in age_str:
            parts = age_str.split("to")
            min_age = float(parts[0].strip())
            max_age = float(parts[1].strip())
            return min_age / 12 if "m" in age_str else min_age, max_age / 12 if "m" in age_str else max_age
        else:
            val = float(age_str.strip())
            return val, val
    except:
        return (None, None)

blv_nutrient_requirements[["age_min", "age_max"]] = blv_nutrient_requirements["age_range"].apply(
    lambda x: pd.Series(parse_age_range(x))
)

# 5. Protein-Spalte korrekt finden und umbenennen
for col in blv_nutrient_requirements.columns:
    if isinstance(col, str) and "Protein" in col and "(g/kg/d)" in col:
        blv_nutrient_requirements.rename(columns={col: "Protein_BLV"}, inplace=True)


# 7. Matching-Funktion
def get_blv_protein(age, gender):
    match = blv_nutrient_requirements[
        (blv_nutrient_requirements["sex"].str.lower() == gender.lower()) &
        (blv_nutrient_requirements["age_min"].astype(float) <= age) &
        (blv_nutrient_requirements["age_max"].astype(float) >= age)
    ]
    if not match.empty:
        return float(match["Protein_BLV"].values[0])
    else:
        return None

# 8. Anwenden auf df_meals
df_meals["Protein_BLV_Ref"] = df_meals.apply(
    lambda row: get_blv_protein(row["Ages"], row["Gender"]), axis=1
)

# 9. Verhältnis berechnen
df_meals["Protein_to_BLV_ratio"] = df_meals["Protein"] / (df_meals["Protein_BLV_Ref"] + 1e-6)

# 10. Vorschau
df_meals[["Ages", "Gender", "Protein", "Protein_BLV_Ref", "Protein_to_BLV_ratio"]].head()

print(blv_nutrient_requirements.columns.tolist())

---

## ✅ **Orientierungswerte für eine gesunde Ernährung**  
(*Idealbereich – für Erwachsene mit normalem Energiebedarf, z. B. 2000–2500 kcal*)

| Feature | Idealwert / Zielbereich | Bedeutung |
|--------|--------------------------|-----------|
| **Protein_per_100kcal** | **> 4.5** | Eiweißdicht – gute Sättigung, Muskelversorgung |
| **Fat_to_Protein_Ratio** | **0.5–1.0** | Ausgewogen: weder zu fett- noch zu eiweißlastig |
| **Carb_to_Fat_Ratio** | **3–5** | Kohlenhydrate überwiegen, aber nicht extrem |
| **Sugar_per_Carb** | **< 0.2** | Max. 20 % der Carbs sollten Zucker sein |
| **Calories_exceed_target** | **0** | Ziel sollte nicht regelmäßig überschritten werden |
| **Sugar_excess** | **0** | Max. 50 g Zucker pro Tag, besser weniger |
| **Protein_deficit_ratio** | **> 1.0** | Mindestens 100 % des Proteinbedarfs decken |
| **Protein_Lunch_vs_Dinner** | **≈ 1.0** | Gleichmäßige Eiweißverteilung über den Tag |
| **Snack_Energy_Share** | **< 0.10** | Snacks max. 10 % der Gesamtkalorien |
| **Carb_ratio_breakfast** | **~ 0.2–0.3** | Ca. 20–30 % der Carbs zum Frühstück |
| **Protein_std** | **< 10** | Gleichmäßige Proteinzufuhr über alle Mahlzeiten |
| **Macronutrient_Diversity** | **5** | Alle 5 Makros vorhanden: Protein, Fett, Carbs, Zucker, Ballaststoffe |

---

### 🧠 Zusammengefasst:  
Ein „gesundes“ Ernährungsprofil wäre z. B.:

- **Viel Protein pro Kalorie** → gute Qualität
- **Wenig Zucker im Verhältnis zu Carbs**
- **Snacks moderat**
- **Verteilung der Makros über den Tag**
- **Kein Kalorienüberschuss**
- **Alle Makronährstoffe dabei**

---


In [ ]:

beispiel_input = {
    # Anteil von Protein pro 100 kcal – zeigt Eiweißdichte der Ernährung
    "Protein_per_100kcal": 5,  # z. B. 5g Eiweiß pro 100 kcal

    # Verhältnis von Fett zu Eiweiß – höher = fettreiche, eiweißarme Ernährung
    "Fat_to_Protein_Ratio": 0.6,  # z. B. 0.6g Fett pro 1g Eiweiß

    # Verhältnis von Kohlenhydraten zu Fett – hoher Wert = viele Carbs, wenig Fett
    "Carb_to_Fat_Ratio": 1.5,  # z. B. 4.5g Carbs pro 1g Fett

    # Anteil Zucker an den gesamten Kohlenhydraten
    "Sugar_per_Carb": 0.4,  # z. B. 40% der Kohlenhydrate sind Zucker

    # 1 = Person hat Kalorienziel überschritten (Daily Calorie Target), 0 = nicht
    "Calories_exceed_target": 1,  # Ja

    # 1 = Person hat mehr als 50g Zucker aufgenommen, 0 = nein
    "Sugar_excess": 0,  # Ja

    # Verhältnis tatsächlicher Proteinzufuhr zur Empfehlung gem. BLV
    "Protein_deficit_ratio": 0.7,  # z. B. nur 70% des benötigten Proteins aufgenommen

    # Verhältnis Protein Mittagessen zu Abendessen – sagt was über Tagesverteilung
    "Protein_Lunch_vs_Dinner": 2.0,  # doppelt so viel Eiweiß zu Mittag

    # Anteil der Snacks an den gesamten Tageskalorien
    "Snack_Energy_Share": 0.12,  # z. B. 12% der Kalorien stammen aus Snacks

    # Anteil der Frühstücks-Carbs an allen Carbs
    "Carb_ratio_breakfast": 0.2,  # 20% der Carbs werden beim Frühstück gegessen

    # Streuung der Proteinwerte über die Mahlzeiten – je höher, desto ungleichmäßiger verteilt
    "Protein_std": 15.0,  # z. B. starke Schwankungen bei Proteinzufuhr über den Tag

    # Wie viele Makronährstoffe wurden überhaupt konsumiert (>0g)?
    "Macronutrient_Diversity": 5  # alle 5 (Protein, Fett, Carbs, Zucker, Ballaststoffe)
}


# Aufruf mit deinem Modell (z. B. clf)
text, proz = vorhersage_chronisches_risiko(beispiel_input, clf, features)

print(f"🧪 Vorhersage: {text} ({proz}% Wahrscheinlichkeit)")


In [ ]:
#Welche Cluster sind eher „gesund“?
df_meals_clean.explode("Disease_List").groupby(["Cluster", "Disease_List"]).size().unstack().fillna(0)


In [ ]:
# Flavia versucht etwas - Clusteranalyse von Ernährungsmustern - Zuteilung von Lebensmitteln an ein bestimmtes Cluster
# KAQ2: Welche Ernährungsmuster sind statistisch singfikant mit bestimmten Krankheitsbildern?
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


# Extrahieren der numerischen Nährstoffspalten
df_meals_numeric = df_meals.select_dtypes(include=['float64', 'int64'])

# Skalierung der Daten
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_meals_numeric)

# K-Means Clustering durchführen (mit 2 Clustern, da du 2 Ernährungsmuster wünschst)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(df_scaled)

# Die Cluster-Zuordnungen der Lebensmittel hinzufügen
df_meals['Cluster'] = kmeans.labels_

# Silhouettenbewertung für die Cluster
silhouette_avg = silhouette_score(df_scaled, kmeans.labels_)

# Ergebnisse anzeigen
print(f"Silhouettenbewertung: {silhouette_avg}")

# PCA durchführen, um die Daten auf 2 Dimensionen zu reduzieren und die Cluster zu visualisieren
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Visualisierung der Cluster basierend auf den PCA-Komponenten
plt.figure(figsize=(8,6))
plt.scatter(df_pca[:, 0], df_pca[:, 1], c=df_meals['Cluster'], cmap='viridis', marker='o', alpha=0.6)
plt.title('Clusteranalyse der Lebensmittel anhand der Nährstoffe (2 Cluster)', fontsize=16)
plt.xlabel('Erste Hauptkomponente (PCA)', fontsize=12)
plt.ylabel('Zweite Hauptkomponente (PCA)', fontsize=12)
plt.colorbar(label='Cluster')
plt.show()

# Eine kleine Zusammenfassung der Cluster anzeigen
print(df_meals[['food', 'Cluster']].head())

# Flavia versucht etwas
# KAQ2: Welche Ernährungsmuster sind statistisch singfikant mit bestimmten Krankheitsbildern?
Die Ergebnisse der Clusteranalyse, bei der die Anzahl der Cluster auf 2 gesetzt wurde, ermöglichen es, zwei verschiedene Ernährungsmuster zu identifizieren. Die genaue Zuordnung dieser Cluster zu bestimmten Ernährungsmustern hängt von den Nährstoffprofilen der Lebensmittel in jedem Cluster ab. Um die beiden Cluster zu interpretieren, schauen wir uns die Nährstoffe in jedem Cluster genauer an und analysieren, welche Arten von Lebensmitteln diese Cluster ausmachen.

**1. Cluster 1: Nahrungsmittel mit hohem Fettgehalt und niedrigem Kohlenhydratanteil**
- **Ernährungsmuster:** Ein Cluster könnte Lebensmittel wie **fettreiche** Lebensmittel enthalten, die wenig Zucker und Kohlenhydrate haben. Dies entspricht häufig Diäten wie der **ketogenen Diät** oder der **low-carb Diät**.

- **Beispiel-Lebensmittel:**

   - Öle (z.B. Olivenöl, Kokosöl)

   - Fettreiche Milchprodukte (z.B. Sahne, Butter)

   - Nüsse und Samen (z.B. Mandeln, Walnüsse)

   - Fleisch und Fisch mit hohem Fettgehalt (z.B. Lachs, fettes Rindfleisch)

**Charakteristika:**

- Hoher Anteil an gesättigten Fetten und Omega-3-Fettsäuren.

- Geringe Mengen an Kohlenhydraten und Zucker.

- Diese Lebensmittel sind reich an Energie und eignen sich gut für Diäten mit niedrigem Kohlenhydratgehalt.

**2. Cluster 2: Nahrungsmittel mit hohem Kohlenhydratanteil und moderatem Fettgehalt**
- **Ernährungsmuster:** Das zweite Cluster könnte Lebensmittel mit einem höheren Anteil an **Kohlenhydraten** und einem moderaten Anteil an Fetten beinhalten. Dies ist typisch für eine **mediterrane Ernährung, vegetarische Ernährung** oder sogar die **DASH-Diät**.

- **Beispiel-Lebensmittel:**

   - Vollkornprodukte (z.B. Haferflocken, Quinoa, Vollkornreis)

   - Hülsenfrüchte (z.B. Linsen, Kichererbsen)

   - Obst und Gemüse

   - Pflanzliche Öle (z.B. Olivenöl)

   - Mageres Fleisch und Fisch (z.B. Hähnchenbrust, Lachs)

**Charakteristika:**

- Hoher Anteil an komplexen Kohlenhydraten und Ballaststoffen.

- Mäßiger Fettgehalt, hauptsächlich gesunde ungesättigte Fette.

- Reich an Vitaminen, Mineralstoffen und Antioxidantien.

**Wie können diese Muster interpretiert werden?**
- **Cluster 1** (z.B. ketogene Diät oder Low-Carb-Diät): Diese Ernährungsmuster betonen einen hohen Fettgehalt und eine niedrige Aufnahme von Kohlenhydraten. Ziel ist es, den Körper in einen Zustand der Ketose zu versetzen, bei dem Fett als Hauptenergiequelle genutzt wird.

- **Cluster 2** (z.B. mediterrane oder pflanzenbasierte Diät): Diese Ernährungsmuster betonen eine ausgewogene Aufnahme von Kohlenhydraten, Fetten und Proteinen, wobei der Fokus auf pflanzlichen Lebensmitteln, gesunden Fetten und Vollwertkost liegt. Diese Diäten sind oft mit langfristiger Gesundheit und Prävention von chronischen Erkrankungen verbunden.

**Was könnte noch hilfreich sein?**
- **Detaillierte Analyse der Cluster:** Um genau zu bestimmen, welche spezifischen Nährstoffe für jedes Cluster charakteristisch sind, könnten wir die Mittelwerte der Nährstoffe innerhalb jedes Clusters berechnen und vergleichen. So könntest du erkennen, welcher Cluster mehr Fette, Kohlenhydrate oder Ballaststoffe enthält.

- **Lebensmittelbeispiele:** Eine detaillierte Liste der Lebensmittel in jedem Cluster könnte dir helfen, zu verstehen, welche Lebensmittel zu welchem Muster gehören.

In [ ]:
# Flavia versucht etwas - Erweiterte Clusteranalyse von Ernährungsmustern
# KAQ2: Welche Ernährungsmuster sind statistisch singfikant mit bestimmten Krankheitsbildern?
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


# Überprüfen der Spaltennamen
print("Spaltennamen im df_meals-Datensatz:")
print(df_meals.columns)

# Extrahieren der numerischen Nährstoffspalten
df_meals_numeric = df_meals.select_dtypes(include=['float64', 'int64'])

# Skalierung der Daten
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_meals_numeric)

# K-Means Clustering durchführen (mit 2 Clustern)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(df_scaled)

# Die Cluster-Zuordnungen der Lebensmittel hinzufügen
df_meals['Cluster'] = kmeans.labels_

# Berechnung der Mittelwerte der Nährstoffe in jedem Cluster
cluster_means = df_meals.groupby('Cluster')[df_meals_numeric.columns].mean()

# Anzeige der Mittelwerte der Nährstoffe in jedem Cluster
print("\nDurchschnittswerte der Nährstoffe pro Cluster:")
print(cluster_means)

# Überprüfen, ob 'Dietary Fiber' in den Mittelwerten vorhanden ist
# Wenn die Spalte vorhanden ist, wird sie einbezogen, andernfalls überspringen wir sie
available_nutrients = ['Fat', 'Carbohydrates', 'Dietary Fiber']
available_nutrients = [nutrient for nutrient in available_nutrients if nutrient in cluster_means.columns]

# Vergleich der Nährstoffe in jedem Cluster (z.B. Fette, Kohlenhydrate, Ballaststoffe)
nutrient_comparison = cluster_means[available_nutrients]
print("\nVergleich der Nährstoffe pro Cluster:")
print(nutrient_comparison)

# Lebensmittelbeispiele aus jedem Cluster
print("\nBeispiele für Lebensmittel aus jedem Cluster:")
for cluster in range(2):
    cluster_foods = df_meals[df_meals['Cluster'] == cluster]['food'].head(10)  # Zeigt die ersten 10 Lebensmittel aus jedem Cluster
    print(f"\nCluster {cluster}:")
    print(cluster_foods)

# Silhouettenbewertung für die Cluster
silhouette_avg = silhouette_score(df_scaled, kmeans.labels_)
print(f"\nSilhouettenbewertung: {silhouette_avg}")

# PCA durchführen, um die Daten auf 2 Dimensionen zu reduzieren und die Cluster zu visualisieren
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Visualisierung der Cluster basierend auf den PCA-Komponenten
plt.figure(figsize=(8,6))
plt.scatter(df_pca[:, 0], df_pca[:, 1], c=df_meals['Cluster'], cmap='viridis', marker='o', alpha=0.6)
plt.title('Clusteranalyse der Lebensmittel anhand der Nährstoffe (2 Cluster)', fontsize=16)
plt.xlabel('Erste Hauptkomponente (PCA)', fontsize=12)
plt.ylabel('Zweite Hauptkomponente (PCA)', fontsize=12)
plt.colorbar(label='Cluster')
plt.show()

# Eine kleine Zusammenfassung der Cluster anzeigen
print("\nZusammenfassung der ersten 5 Lebensmittel und ihrer Clusterzuordnung:")
print(df_meals[['food', 'Cluster']].head())



Im Vergleich zur ersten Clusteranalyse, die lediglich die Identifikation von zwei Ernährungsmustern mittels K-Means und deren Visualisierung anhand von PCA beinhaltete, geht der erweiterte Code einen Schritt weiter. Neben der Clusterbildung und der Visualisierung wird nun eine detaillierte Analyse der Nährstoffzusammensetzung innerhalb der beiden Cluster durchgeführt. Ziel der erweiterten Analyse ist es, nicht nur die Cluster zu identifizieren, sondern auch die durchschnittlichen Nährstoffwerte in jedem Cluster zu berechnen und die Unterschiede zwischen den Ernährungsmustern genauer zu verstehen.

In [ ]:
# Flavia versucht etwas - Erweiterte Clusteranalyse von Ernährungsmustern kombiniert mit Übergewicht, Bluthochdruck und Diabetes --> das macht keinen Sinn, muss es nochmals überarbeiten
# KAQ2: Welche Ernährungsmuster sind statistisch singfikant mit bestimmten Krankheitsbildern?
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Extract relevant nutrient columns
df_meals_numeric = df_nutrition[['Protein', 'Sugar', 'Sodium', 'Calories', 'Carbohydrates', 'Fiber', 'Fat']]

# Scale the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_meals_numeric)

# Perform K-Means Clustering (with 2 clusters)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(df_scaled)

# Add cluster assignments to the dataframe
df_nutrition['Cluster'] = kmeans.labels_

# Check how diseases are distributed in clusters
df_nutrition['Weight Gain'] = df_nutrition['Disease'].str.contains('Weight Gain')
df_nutrition['Hypertension'] = df_nutrition['Disease'].str.contains('Hypertension')
df_nutrition['Diabetes'] = df_nutrition['Disease'].str.contains('Diabetes')

# Group by Cluster and count disease occurrences
disease_counts = df_nutrition.groupby('Cluster')[['Weight Gain', 'Hypertension', 'Diabetes']].sum()

# Data for the updated stacked bar chart
cluster_labels_named = ['Energie- und Nährstoffreiche Ernährung', 'Mäßig Nährstoffreiche Ernährung']
weight_gain_named = disease_counts['Weight Gain']
hypertension_named = disease_counts['Hypertension']
diabetes_named = disease_counts['Diabetes']

# Create the updated stacked bar chart with disease legend and proper labeling for the "Anzahl der Fälle"
fig, ax = plt.subplots(figsize=(10, 6))

# Increase the bar width and adjust spacing
bar_width = 0.4  # Adjust the bar width for better fit

# Create the stacked bars
ax.bar(cluster_labels_named, weight_gain_named, width=bar_width, label='Übergewicht (Weight Gain)', color='skyblue')
ax.bar(cluster_labels_named, hypertension_named, bottom=weight_gain_named, width=bar_width, label='Bluthochdruck (Hypertension)', color='orange')
ax.bar(cluster_labels_named, diabetes_named, bottom=weight_gain_named + hypertension_named, width=bar_width, label='Diabetes', color='green')

# Add labels and title with proper descriptions
ax.set_xlabel('Ernährungsmuster (Cluster)', fontsize=12)
ax.set_ylabel('Anzahl der Fälle', fontsize=12)
ax.set_title('Verteilung von Krankheiten in den Ernährungsmustern', fontsize=14)

# Add the standard disease legend inside the plot
ax.legend(title="Krankheiten", loc='upper left', bbox_to_anchor=(1, 1))

# Adding descriptions of the clusters below the plot with more space
plt.figtext(0.5, -0.25,
            'Energie- und Nährstoffreiche Ernährung: Hoher Protein-, Zucker-, Kalorien- und Fettgehalt\n'
            'Mäßig Nährstoffreiche Ernährung: Geringer Protein-, Zucker- und Fettgehalt',
            ha='center', va='top', fontsize=12)

# Adjust layout to give more space for descriptions and prevent overlap
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Increase the right margin for the legend

# Show the plot
plt.show()



**Beschreibung des Balkendiagramms:**

Das Diagramm zeigt die Verteilung von Übergewicht, Bluthochdruck und Diabetes in zwei verschiedenen Ernährungsmustern (Clustern), die durch eine Clusteranalyse basierend auf Nährwertdaten gebildet wurden.

**„Energie- und Nährstoffreiche Ernährung“:** In diesem Cluster leiden insgesamt **810 Personen** an mindestens einer der drei Krankheiten.

**„Mäßig Nährstoffreiche Ernährung“:** In diesem Cluster sind insgesamt **1212 Personen** betroffen.

Die **Balken** im Diagramm sind **gestapelt**, wobei die Höhe jedes Balkens die **Anzahl der betroffenen Personen** darstellt. Jeder Balken ist in drei Farben unterteilt:

- **Blau** für Übergewicht (Weight Gain),

- **Orange** für Bluthochdruck (Hypertension),

- **Grün** für Diabetes.

**Gesamtzahl der Personen im Datensatz: 1698 Personen.**

Das Diagramm hilft zu verstehen, wie die Verteilung von Krankheiten in den beiden Ernährungsmustern aussieht und wie viele Personen insgesamt betroffen sind.

---

**Erklärung des Codes:**

Der Code, führt eine Clusteranalyse von Ernährungsmustern durch und untersucht dann, wie diese Muster mit bestimmten Krankheitsbildern (wie Übergewicht, Bluthochdruck und Diabetes) zusammenhängen. Hier ist eine einfache Erklärung, wie der Code funktioniert:

**1. Datenvorbereitung:**

Nährstoffspalten extrahieren: Zuerst werden die Nährstoffdaten (z.B. Protein, Zucker, Kalorien, Natrium, Kohlenhydrate, Fett und Ballaststoffe) aus dem Datensatz extrahiert.

**2. Skalierung der Daten:**

Da Nährstoffe unterschiedliche Skalen haben (z.B. Kalorien sind viel größer als Zucker), wird der StandardScaler verwendet, um die Daten zu skalieren. Das bedeutet, dass jede Nährstoffspalte so umgerechnet wird, dass sie einen Mittelwert von 0 und eine Standardabweichung von 1 hat.

**3. Durchführung der Clusteranalyse (K-Means):**

Jetzt wird eine K-Means Clusteranalyse durchgeführt, um die Daten in zwei Gruppen (Cluster) zu unterteilen.

**4. Krankheitsinformationen hinzufügen:**

Der Code überprüft, ob bestimmte Krankheiten (Übergewicht, Bluthochdruck und Diabetes) in der Krankheitsliste enthalten sind und erstellt neue Spalten für jede dieser Krankheiten.
- Diese neuen Spalten haben den Wert True, wenn die Krankheit in der Krankheitsbeschreibung für eine bestimmte Person enthalten ist, und False, wenn nicht.

**5. Gruppierung der Daten nach Cluster und Zählen der Krankheitsfälle:**

Die Daten werden nun nach den Clustern gruppiert, und für jedes Cluster wird gezählt, wie viele Personen an den jeweiligen Krankheiten (Übergewicht, Bluthochdruck, Diabetes) leiden.

**6. Visualisierung der Ergebnisse:**
Schließlich wird ein gestapeltes Balkendiagramm erstellt, das zeigt, wie die Krankheitsfälle in den beiden Clustern verteilt sind.

**Fazit:**

- Der K-Means-Algorithmus teilt die Ernährungsmuster in zwei Cluster auf. Diese Cluster werden dann mit den Krankheiten Übergewicht, Bluthochdruck und Diabetes in Verbindung gesetzt.

- Du kannst sehen, ob und wie häufig diese Krankheiten in den beiden Clustern auftreten, was dir hilft, festzustellen, ob bestimmte Ernährungsmuster mit den Krankheiten in Verbindung stehen.

In [ ]:
# Flavia versucht etwas - Erweiterte Clusteranalyse von Ernährungsmustern kombiniert mit Übergewicht, Bluthochdruck und Diabetes --> das macht keinen Sinn, muss es nochmals überarbeiten
# KAQ2: Welche Ernährungsmuster sind statistisch singfikant mit bestimmten Krankheitsbildern?

import matplotlib.pyplot as plt

# Definieren der Krankheiten, die wir im Diagramm berücksichtigen wollen
disease_keywords = ['Diabetes', 'Hypertension', 'Weight Gain']

# Erstellen einer neuen Spalte, die für jede Zeile die relevanten Krankheiten enthält
def disease_category(disease):
    categories = {'Diabetes': 0, 'Hypertension': 0, 'Weight Gain': 0}
    for keyword in disease_keywords:
        if keyword in disease:
            categories[keyword] = 1
    # Rückgabe der Kombination der Krankheiten (Diabetes, Bluthochdruck, Übergewicht)
    return tuple(categories.values())

# Wenden der Funktion auf die Disease-Spalte im df_nutrition an
df_nutrition['Disease Category'] = df_nutrition['Disease'].apply(disease_category)

# Zählen der Häufigkeiten jeder Kategorie
category_counts = df_nutrition['Disease Category'].value_counts()

# Umbenennen der Kategorien in verständliche Labels
category_labels = {
    (1, 0, 0): "Diabetes",
    (0, 1, 0): "Bluthochdruck",
    (0, 0, 1): "Übergewicht",
    (1, 1, 0): "Diabetes & Bluthochdruck",
    (1, 0, 1): "Diabetes & Übergewicht",
    (0, 1, 1): "Bluthochdruck & Übergewicht",
    (1, 1, 1): "Diabetes, Bluthochdruck & Übergewicht",
}

# Wir müssen jetzt alle Personen ohne diese Krankheiten in die Kategorie "Keine dieser Krankheiten" zählen
# Füge Personen hinzu, die keine dieser drei Krankheiten haben
category_counts[(0, 0, 0)] = len(df_nutrition[df_nutrition['Disease Category'] == (0, 0, 0)])

# Umbenennen der Kategorien in verständliche Labels
category_counts.index = category_counts.index.map(category_labels)
category_labels[(0, 0, 0)] = "Keine dieser Krankheiten"

# Berechnung der Prozentwerte
total = category_counts.sum()
percentages = (category_counts / total) * 100

# Ausgabe der Zählung und der Prozentsätze, um sicherzustellen, dass sie korrekt sind
print("Häufigkeiten der Kategorien:")
print(category_counts)
print("\nProzentuale Verteilung:")
print(percentages)

# Erstellen des Kreisdiagramms ohne innere Beschriftungen und ohne Labels
plt.figure(figsize=(8, 8))

# Plotten ohne innere Beschriftungen und ohne Labels
ax = category_counts.plot(kind='pie', autopct=None, startangle=90, colors=plt.cm.Paired.colors, labels=None)

# Titel und Y-Achsen-Beschriftung entfernen
plt.title('Verteilung der Krankheiten (Diabetes, Bluthochdruck, Übergewicht)')
plt.ylabel('')  # Entfernen der Y-Achsen-Beschriftung

# Erstellen der Legende mit absoluten Zahlen und Prozentwerten
legend_labels = [
    f"{label}: {count} ({percent:.1f}%)"
    for label, count, percent in zip(category_labels.values(), category_counts, percentages)
]

# Manuelles Hinzufügen der Legende
plt.legend(title='Krankheiten', labels=legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))

# Diagramm anzeigen
plt.show()




In [ ]:
# =====================================
# Schritt 5: Analyse KAQ
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
# =====================================
# Schritt 5: Analyse KAQ
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
# =====================================
# Schritt 6: Analyse KAQ
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
# =====================================
# Schritt 7: Analyse KAQ
# =====================================

# 🧠 Frage: xxxxxx
# 🎯 Ziel: xxxxxx

In [ ]:
 # Ab hier macht murat was

In [ ]:
import pandas as pd
import plotly.express as px

# 1. Histogramm – Kalorienverteilung
px.histogram(df_meals, x="Calories", nbins=30, title="Verteilung der Tageskalorien").show()

# 2. Boxplot – Zuckeraufnahme nach Geschlecht
px.box(df_meals, x="Gender", y="Sugar", title="Zuckeraufnahme nach Geschlecht").show()

# 3. Scatterplot – Protein vs. Zucker nach Krankheit
px.scatter(
    df_meals,
    x="Protein",
    y="Sugar",
    color="Disease",
    title="Protein vs. Zuckeraufnahme nach Krankheit",
    hover_data=["Ages", "Gender"]
).show()

# 4. Violinplot – Fettzufuhr je Aktivitätslevel
px.violin(
    df_meals,
    x="Activity Level",
    y="Fat",
    box=True,
    points="all",
    title="Fettzufuhr je Aktivitätslevel"
).show()

Super, danke fürs Teilen des Screenshots! Ich sehe sofort, was du meinst. Hier kommt die Analyse deiner **Scatterplot-Visualisierung (Protein vs. Zucker) mit Trendlinie**:

---

### 📊 **Was zeigt dieser Plot?**

- **X-Achse:** Proteinaufnahme pro Tag (in g)
- **Y-Achse:** Zuckeraufnahme pro Tag (in g)
- **Jeder Punkt:** Eine Person aus dem Datensatz
- **Farben:** Krankheit(en) der jeweiligen Person
- **Gerade Linie:** Eine **Trendlinie (Regression)** → zeigt den **durchschnittlichen Zusammenhang**

---

### 📈 **Interpretation der steigenden Trendlinie**

- Die **positive Steigung** der Linie bedeutet:  
  **Mehr Protein → tendenziell auch mehr Zuckeraufnahme.**

- Das kann ein Hinweis sein auf:
  - Allgemein **höheren Energiebedarf** (z. B. sportliche oder junge Personen)
  - Ernährungsmuster, bei denen **Makros insgesamt steigen** (z. B. durch große Portionen oder energie-dichte Lebensmittel)

---

### 🩺 **Bezug zu Krankheiten**

- Der Datensatz zeigt:
  - Auch Personen mit mehreren Krankheiten (z. B. *Diabetes*, *Heart Disease*) liegen in diesem Bereich mit **hoher Protein- und Zuckeraufnahme**.
  - Diese Ballung könnte für **KAQ 1 und KAQ 3** interessant sein:
    > Gibt es Cluster mit hohem Zucker & Protein, die **häufiger krank** sind?

---

### 🧠 **Ideen für weiterführende Schritte:**

1. **Clustering:** Nutze K-Means oder DBSCAN, um Gruppen mit ähnlichem Makroprofil zu finden.
2. **Split nach Krankheit:** Mache den gleichen Scatterplot **nur für Diabetiker** vs. **Nicht-Diabetiker**.
3. **Korrelation berechnen:** z. B. Pearson- oder Spearman-Korrelation zwischen Protein & Zucker.
4. **Deckungsgrade:** Wie gut erfüllen diese Gruppen die BLV-Empfehlungen?

---

Wenn du willst, kann ich dir den Code liefern, um das alles direkt im Notebook umzusetzen. Oder ich rechne dir als Nächstes die Korrelation aus – sag einfach, was dir lieber ist.

In [ ]:
# 📦 Importiere notwendige Module
import os
from nbformat import read
from nbformat.validator import validate

# 📍 Definiere den vollständigen Pfad zu deinem Notebook
notebook_path = r"onlyAnalysis_Ernährung.ipynb"

# 🧪 Überprüfe, ob die Datei existiert
if not os.path.exists(notebook_path):
    print(f"⚠️ Datei nicht gefunden: {notebook_path}")
    print("Bitte überprüfen Sie den Pfad und stellen Sie sicher, dass die Datei existiert.")
    # Exit the script gracefully
    import sys
    sys.exit(1)
    # 🗂️ Zeige den absoluten Pfad der Datei
    absolute_path = os.path.abspath(notebook_path)
    print(f"📂 Absoluter Pfad der Datei: {absolute_path}")
# 📖 Öffne das Notebook und lade den Inhalt
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = read(f, as_version=4)

# ✅ Validierung durchführen
validate(nb)
print("✅ Das Notebook ist gültig und im richtigen Format!")